# Analysis of simulated data


TODO add here a brief description of your project or analysis. Explain the purpose, goals, or context of the notebook here.

## Table of Contents
- [1. Chapter 1 - Introduction](#chapter-1-introduction)
- [2. Chapter 2 - R packages](#chapter-2-R-packages)
- [3. Chapter 3 - Input specification](#chapter-3-Input-specification)
- [4. Chapter 4 - Supermatrix and Supertree trees VS true topologies](#chapter-4-Supermatrix-and-Supertree-trees-VS-true-topologies)
- [5. Chapter 5 - BigTree VS true topologies](#chapter-5-BigTree-VS-true-topologies)
- [6. Chapter 6 - RF distance summaries](#chapter-6-RF-distance-summaries)
    - [6.1 Subchapter 6.1 - Table 1](#subchapter-6_1-Table-1)
    - [6.2 Subchapter 6.2 - Figure 2](#subchapter-6_2-Figure-2)
- [7. Chapter 7 - Running times](#chapter-7-Running-times)
    - [7.1 Subchapter 7.1 - Figure 3](#subchapter-7_1-Figure-3)
- [8. Chapter 8 - Bootstrap analysis](#chapter-8-Bootstrap-analysis)
    - [8.1 Subchapter 8.1 - AUC computation](#subchapter-8_1-AUC-computation)
    - [8.2 Subchapter 8.2 - MCC computation](#subchapter-8_2-MCC-computation)
    - [8.3 Subchapter 8.3 - Reference presence vs BS ](#subchapter-8_3-Reference-presence-vs-BS)

<a id="chapter-1-introduction"></a>

## Chapter 1 - Introduction

TODO add description of notebook and how to use it.


<a id="chapter-2-R-packages"></a>

## Chapter 2 - R packages

Here is the list of packages needed for this notebook.

In [1]:
library("phangorn")
library("ggplot2")
library("reshape2")
library("geiger")
library("adephylo")
library("phytools")
library("rlist")
library("plotrix")
library("plyr")
library("dplyr")
library("hrbrthemes")
library("rstatix")
library("gplots")
library("viridis")
library('gtools')
library(gridExtra)
library(grid)
library("ggforce")
library("scales")
library("pROC")
library("tidyr")
library("stringr")

Loading required package: ape

Loading required package: phytools

Loading required package: maps

Loading required package: ade4


Attaching package: ‘plotrix’


The following object is masked from ‘package:phytools’:

    rescale



Attaching package: ‘plyr’


The following object is masked from ‘package:maps’:

    ozone



Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from ‘package:ape’:

    where


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘rstatix’


The following objects are masked from ‘package:plyr’:

    desc, mutate


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘gplots’


The following object is masked from ‘package:plotrix’:

    plotC

TODO make a chapter for aqll custom functions

<a id="chapter-3-Input-specification"></a>

## Chapter 3 - Input specification

Change the parent_of_all_dir variable to give the location of the input. All input files are expected to be under it. Otherwise change the single input location to you liking.<br>
All input files or directory paths are listed here.

In [2]:
# the location of the directory that contains all inputs for analyses 
parent_of_all_dir = "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/"

# The directory (one or more) that contain all the results from the main.nf pipeline run. It can be a glob path.
dir_containing_all_pipe_results = paste0(parent_of_all_dir, "rerun_25_species_corrected_branches/rerun_sim_b_factor*")

#  TODO describe what this file does
reference_tree_codefile = file.path(parent_of_all_dir, "ref_tree.code")

# get the species/organism list of names used by the main.nf pipeline
species_name_file = file.path(parent_of_all_dir, "orthologs_org_ids_to_concatenate")


# each run of main.nf has one output directory. here we get that or multiple runs outputs directories.
run_dirs = Sys.glob(dir_containing_all_pipe_results)
run_dirs = run_dirs[file.info(run_dirs)$isdir]

# Create the 'figures' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("figures")) {
    dir.create("figures")
}

# Create the 'tables' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("tables")) {
    dir.create("tables")
}

<a id="chapter-4-Supermatrix-and-Supertree-trees-VS-true-topologies"></a>

## Chapter 4 - Supermatrix and Supertree trees VS true topologies

TODO add description 

In [3]:
# setting global lists that eill hold all family level values
RF_avg_list = list()
collect_branch_len_supermatrix_ml = c()
collect_diameter_supermatrix_ml = c()

for (run in run_dirs) {
    print(basename(run))

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # get al the family subdirectories in each mqin.nf output run dir
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = 0

    # this will contain the mean branch length for all true/reference paralog trees for each gene family
    avg_branch_length = c()

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # get all units trees for all combination of methods 
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 10 == 0) {
            print(m)
        }

        # Get all units trees (250 = 25 organism * 10 samples/statistical repeats) for the combination Minimun Evolution (ME) + supermatrix method (SM). 
        # We unroot every tree and fully resolve it to use the RF.dist package and others. 
        all_units_file  = file.path(m, "results/rerun", "all_units.nwk")
        all_units_trees = read.tree(all_units_file, keep.multi=TRUE)
        all_units_trees = unroot(all_units_trees)
        
        # check if there any non-bynary (consensus trees) among the 250 trees present in the file above
        if (length(which(is.binary(all_units_trees) == FALSE)) != 0) {
            
            # if there are non-binary trees, iterate through them and transformed into a binary one using multi2di package
            for (tree in which(is.binary(all_units_trees) == FALSE)) {
                all_units_trees[[tree]] = multi2di(all_units_trees[[tree]])
            }
        }

        # Get all units trees (250) ME + supertree (ST) method and handle non-binary trees as above
        all_superfine_file  = file.path(m, "results/rerun", "all_units_superfine_trees.nwk")
        all_superfine_trees = read.tree(all_superfine_file, keep.multi=TRUE)
        all_superfine_trees = unroot(all_superfine_trees)
        if (length(which(is.binary(all_superfine_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_superfine_trees) == FALSE)) {
                all_superfine_trees[[tree]] = multi2di(all_superfine_trees[[tree]])
            }
        }

        # Get all units trees (250) Maximum Likelihood (ML) + SM 
        all_units_raxml_file  = file.path(m, "results/rerun", "all_units_RAxML_trees.nwk")
        all_units_raxml_trees = read.tree(all_units_raxml_file, keep.multi=TRUE)
        all_units_raxml_trees = unroot(all_units_raxml_trees)
        if (length(which(is.binary(all_units_raxml_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_units_raxml_trees) == FALSE)) {
                all_units_raxml_trees[[tree]] = multi2di(all_units_raxml_trees[[tree]])
            }
        }

        # take only the last 10 of the 250 above trees (unit25's 10 samples/replicates)(unit25 = all 25 single units concatenated) and collect branch lengths (each branch in tree, 27 in total 2*15-3) and diameter (using patristic distance)
        for(mytree in 241:length(all_units_raxml_trees)) {
            collect_branch_len_supermatrix_ml = c(collect_branch_len_supermatrix_ml, all_units_raxml_trees[[mytree]]$edge.length)
            collect_diameter_supermatrix_ml = c(collect_diameter_supermatrix_ml, max(distTips(all_units_raxml_trees[[mytree]], method = 'patristic')))
        }

        # Get all units trees (250) ML + ST
        all_raxml_superfine_file  = file.path(m, "results/rerun", "all_units_RAxML_superfine_trees.nwk")
        all_raxml_superfine_trees = read.tree(all_raxml_superfine_file, keep.multi=TRUE)
        all_raxml_superfine_trees = unroot(all_raxml_superfine_trees)
        if (length(which(is.binary(all_raxml_superfine_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_raxml_superfine_trees) == FALSE)) {
                all_raxml_superfine_trees[[tree]] = multi2di(all_raxml_superfine_trees[[tree]])
            }
        }

        # Reference tree block, each gene family has it's own reference tree that is the one generated by the simulation step. Along this tree all sequences are generated and then alligned and passed as input for main.nf pipeline.
        ref_tree_file = file.path(run, paste0(basename(m), ".nwk"))
        ref_tree      = read.tree(ref_tree_file, keep.multi=FALSE)

        # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
        # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of omne species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
        ref_subtree   = keep.tip(ref_tree, as.character(ref_codefile$V1))

        # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
        for (j in 1:length(ref_subtree$tip.label)) { 
            ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
        }

        # Unroot the tree and perform aother operation and then save the paralog tree to the specifc gene faimily directory
        ref_subtree        = unroot(ref_subtree)
        ref_subtree        = list(ref_subtree)
        class(ref_subtree) = "multiPhylo"
        subtree_out_file   = file.path(m, "results/rerun", paste0("ref_subtree_", basename(m), ".nwk"))
        # TODO remove this comment write.tree(ref_subtree, file=subtree_out_file)

        # update on the average branch length with the mean of the true/refernce paralog tree of the given gene family
        avg_branch_length = c(avg_branch_length, mean(ref_subtree[[1]]$edge.length))

        # Make a list containing all trees gathered so far and compute all possible pair comparisons among trees. There are 1001*1001-1001 unique pairewise comparisons ( a square matrix of size 1001 with 0 in diagonal ).
        # On this matrix (plotdata) all other measuramnts are done. This is a symmetric matrix (distance matrix)
        all_trees        = c(ref_subtree, all_units_trees, all_superfine_trees, all_units_raxml_trees, all_raxml_superfine_trees)
        class(all_trees) = "multiPhylo"
        all_vs_all_rf    = RF.dist(all_trees, normalize = TRUE)
        plotdata         = data.matrix(all_vs_all_rf)

        # all ME metrics to be computed superfine = SuperTree
        mean_rf           = c()
        mean_superfine_rf = c()

        # all ML metrics to be computed
        mean_raxml_rf           = c()
        mean_raxml_superfine_rf = c()

        # proceed throught the plotdata matrix taking from the second element one every ten up to 242. Basically proceed at chunkcs of 10 (samples). On those chuncks compute the mean and the Standard error. 
        # average all sample "values" whithin a unit. going from 250 value to 25 averages
        # This iterate only through the values of the comparison between the reference paralofg tree (ref_subtree)(first column/row of matrix) and the 250 paraogs trees of ME + SM (all_units_trees)
        for (i in seq(2, (length(orgs$V1)*10)+1, by=10)) {
            mean_rf = c(mean_rf, mean(plotdata[1, i:(i+9)]))
        }

        # This instead goes from row 252 to 492 of the plotdata matrix. It still goes in chuncks of 10 (samples). 
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ME + ST (all_superfine_trees)
        for (i in seq( (length(orgs$V1)*10)+2, (2*(length(orgs$V1)*10)+1), by=10 )) {
            mean_superfine_rf = c(mean_superfine_rf, mean(plotdata[1,i:(i+9)]))
        }

        # This for loop goes from 502 to 742. It still goes in chuncks of 10 (samples).
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ML + SM (all_units_raxml_trees)
        for (i in seq( (2*(length(orgs$V1)*10))+2, (3*(length(orgs$V1)*10)+1), by=10 )) {
            mean_raxml_rf = c(mean_raxml_rf, mean(plotdata[1,i:(i+9)]))
        }

        # This for loop goes from 752 to 992. It still goes in chuncks of 10 (samples).
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ML + ST (all_raxml_superfine_trees)
        for (i in seq( (3*(length(orgs$V1)*10))+2, (4*(length(orgs$V1)*10)+1), by=10)) {
            mean_raxml_superfine_rf = c(mean_raxml_superfine_rf, mean(plotdata[1,i:(i+9)]))
        }

        # collect all mean RF values 25 mean_rf, 25 mean_superfine_rf ecc.. for each family. One for each unit.
        collection_mean_rf      = c(mean_rf, mean_superfine_rf, mean_raxml_rf, mean_raxml_superfine_rf)
  
        # Finally update the "global" lists created at the beginning
        RF_avg_list      = list.append(RF_avg_list, collection_mean_rf)

    }
}



[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_010.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_020.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_030.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_1

In [4]:
# transform the list of lists (RF_avg_list) into a dataframe (matrix) size 500 * 100 (25 units for all combinations of ME,ML and SM,ST)
RF_avg_as_df = ldply(RF_avg_list, recursive=FALSE)

# create the column names for the later dataframe, ME = minimum Evolutio, ML = Maximum Likelihood, SM = SuperMatrix , ST = SuperTree. names will be like ME_SM_unit_1
units_names = c()
for (i in 1:length(orgs$V1)) {
    units_names = c(units_names, paste("ME_SM", "unit", i, sep="_"), 
                    paste("ME_ST", "unit", i, sep="_"), 
                    paste("ML_SM", "unit", i, sep="_"), 
                    paste("ML_ST", "unit", i, sep="_"))
}

# natural_sort them or human readable sort, so that the order is  ME_SM unit1...unit25, ME_ST unit1....uniy25, ML_SM and ML_ST
units_names = mixedsort(units_names)

# TODO see if standar error se was actually usefull
RF_df = round(RF_avg_as_df, digits=3)

# add the units name generated before as column names for the dataframe and save it to file
colnames(RF_df) = units_names
write.table(RF_df, file = "tables/avg_RF_between_ref_supermatrix_and_supertree.tsv", quote = FALSE, row.names = FALSE)

<a id="chapter-5-BigTree-VS-true-topologies"></a>

## Chapter 5 BigTree VS true topologies

TODO add description

In [6]:
# Trees from BigTree (full_aln) ME/ML (paralog tree) VS true topologies 

paralog_ME_vs_true_tree = c()
paralog_ML_vs_true_tree = c()
RF_matrices = list()


collect_branch_len_sub_from_ref = c()
collect_diameter_sub_from_ref = c()

for (run in run_dirs) {
    print(basename(run))

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # get al the family subdirectories in each mqin.nf output run dir
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = 0

    # this will contain the mean branch length for all true/reference paralog trees for each gene family
    avg_branch_length = c()

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # extract and work on all BigTree paralog files for all families. this trees are computed from the main.nf pipeline in simulation mode.
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 10 == 0) {
            print(m)
        }

        # read the paralog tree obtained from the BigTree (gene family tree) created unsing ME.
        # the paralog tree was obtained extracting each species subtree for the gene family tree, then this (25) subtrees were merged into one exactly like is done for SuperTree (using superfine).
        fastme_paralog_tree_file          = file.path(m, "results/rerun", paste0(basename(m), "_full_aln_coded.phylip_fastme_tree_paralog_tree.nwk"))
        fastme_paralog_tree_from_full_aln = read.tree(fastme_paralog_tree_file, keep.multi=FALSE)
        fastme_paralog_tree_from_full_aln = unroot(fastme_paralog_tree_from_full_aln)
        fastme_paralog_tree_from_full_aln = list(fastme_paralog_tree_from_full_aln)


        # read the paralog tree obtained from the BigTree (gene family tree) created unsing ML. paralog tree constructed same way as above.
        raxml_paralog_tree_file           = file.path(m, "results/rerun", paste0("RAxML_bestTree.", basename(m), "_full_aln_coded_raxml_paralog_tree.nwk"))   
        raxml_paralog_tree_from_full_aln  = read.tree(raxml_paralog_tree_file, keep.multi=FALSE)
        raxml_paralog_tree_from_full_aln  = unroot(raxml_paralog_tree_from_full_aln)
        raxml_paralog_tree_from_full_aln  = list(raxml_paralog_tree_from_full_aln)
                
        # read the reference tree (gene family tree) the one set as thruth and on which the sequences were generated along.
        ref_tree_file = file.path(run, paste0(basename(m), ".nwk"))
        ref_tree      = read.tree(ref_tree_file, keep.multi=FALSE)

        # each reference species paralog tree is identical in topologty (designed so) but diverges for branch lengths. here we retrieve some info for that.
        for (species in orgs$V1) {
            
            # ectract tips like before and then extract subtree and unroot it. 
            # this is the refence paralog tree for each species and whithin one family all species have the same paralog tree as far as topology goes, what changes between this species paralog trees is their branch lengths.
            tips_to_keep = grep(paste0("^", species), ref_tree$tip.label, perl = TRUE)
            test_subtree = keep.tip(ref_tree, tips_to_keep)
            test_subtree = unroot(test_subtree)

            # get branch length and diameter
            collect_branch_len_sub_from_ref = c(collect_branch_len_sub_from_ref, test_subtree$edge.length)
            collect_diameter_sub_from_ref   = c(collect_diameter_sub_from_ref, max(distTips(test_subtree, method = 'patristic')))
        }

        # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
        # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of one species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
        ref_subtree   = keep.tip(ref_tree, as.character(ref_codefile$V1))

        # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
        for (j in 1:length(ref_subtree$tip.label)) { 
            ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
        }

        # Unroot the tree and put into a list
        ref_subtree        = unroot(ref_subtree)
        ref_subtree        = list(ref_subtree)


        # update on the average branch length with the mean of the true/refernce paralog tree of the given gene family
        avg_branch_length = c(avg_branch_length, mean(ref_subtree[[1]]$edge.length))
        
        # put all  BIgTree (ME/ML) paralog tres (2) and the reference paralog tree into one list.
        all_trees        = c(ref_subtree, fastme_paralog_tree_from_full_aln, raxml_paralog_tree_from_full_aln)
        class(all_trees) = "multiPhylo"
        
        # compute the RF dictance between 1 refernce paralog the other two paralog trees: ME ML. In reality all difrrences between them.
        all_vs_all_rf = RF.dist(all_trees, normalize=TRUE)
        plotdata      = data.matrix(all_vs_all_rf)
        RF_matrices   = list.append(RF_matrices, plotdata)
        
        # do the average of all pair comparison between the species paralog trees and the reference tree. -c[1] means not consider the first column, because first row first column is RF between reference tree and itself
        paralog_ME_vs_true_tree = c(paralog_ME_vs_true_tree, plotdata[1, 2])
        paralog_ML_vs_true_tree = c(paralog_ML_vs_true_tree, plotdata[1, 3])
    }
}

write.table(paralog_ME_vs_true_tree, file = "tables/paralog_ME_vs_true_tree.tsv", quote=FALSE, row.names = FALSE, col.names = FALSE)
write.table(paralog_ML_vs_true_tree, file = "tables/paralog_ML_vs_true_tree.tsv", quote=FALSE, row.names = FALSE, col.names = FALSE)

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_010.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_020.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_030.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_1

<a id='chapter-6-RF-distance-summaries'></a>

## Chapter 6 - RF distance summaries

todo add description

<a id='subchapter-6_1-Table-1'></a>

### Subchapter 6.1 - Table 1
todo add description.

In [7]:
# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/avg_RF_between_ref_supermatrix_and_supertree.tsv", header=T)

# do the mean and sd of the column. effectively getting a vector of size 100 filed with averages of averages and with ids like SM_ML_unit9
rf_df_new_means = colMeans(df_new)
sd_df_new_means = apply(df_new, 2, sd)

# Initialize empty list to hold the extracted method names and unit names values
methods = c("ME_SM", "ME_ST", "ML_SM",  "ML_ST")
units   = 25

# Create an empty dataframe with row names for methods and columns for unit_1 to unit_25
avg_rf_df_method_unit           = data.frame(Method = methods, matrix(ncol = units, nrow = 4))
colnames(avg_rf_df_method_unit) = c("Method", paste0("unit_", 1:units))

# Fill the dataframe using the keys of the named vector of rf_df_new_means, these names are the same for the vector sd_df_new_means too.
for (key in names(rf_df_new_means)) {

  # Split the key into parts (method and unit)
  parts  = strsplit(key, "_unit_")[[1]]
  method = parts[1]
  unit   = as.numeric(parts[2])

  # Place the value in the correct row and column of the dataframe adding the standard deviation to it.
  avg_rf_df_method_unit[avg_rf_df_method_unit$Method == method, paste0("unit_", unit)] = paste0(round(rf_df_new_means[key], digit=2), "  sd  ", round(sd_df_new_means[key], digit=2))
}

# create the actual table with only unit1, unit25 difference values from above df
diff   = c(as.numeric(sub(" sd.*", "", avg_rf_df_method_unit$unit_1)) - as.numeric(sub(" sd.*", "", avg_rf_df_method_unit$unit_25)))
table1 = data.frame(Method = avg_rf_df_method_unit$Method,
                    "Unit 25"    = avg_rf_df_method_unit$unit_25,
                    "Unit 1"     = avg_rf_df_method_unit$unit_1,
                    "Difference" = diff )

# read the table compiled in chapter 5. So this cell does not depend on that one to run, it just needs the table file.
df_bigtree_ME = read.table("tables/paralog_ME_vs_true_tree.tsv", header=F)
df_bigtree_ML = read.table("tables/paralog_ML_vs_true_tree.tsv", header=F)

# make average of all family BigTree RF (ME and ML) distances to their respective paralog tree. going from 500 values to 1.
rf_df_bigtree_ME_mean = colMeans(df_bigtree_ME)
rf_df_bigtree_ME_sd   = apply(df_bigtree_ME, 2, sd)
rf_df_bigtree_ML_mean = colMeans(df_bigtree_ML)
rf_df_bigtree_ML_sd   = apply(df_bigtree_ML, 2, sd)

# add the above four values to table1
bigtree_ME_row = c("ME_BigTree", paste0(round(rf_df_bigtree_ME_mean, digits = 2), "  sd  ", round(rf_df_bigtree_ME_sd, digits = 2)), "Na", "Na")
bigtree_ML_row = c("ML_BigTree", paste0(round(rf_df_bigtree_ML_mean, digits = 2), "  sd  ", round(rf_df_bigtree_ML_sd, digits = 2)), "Na", "Na")
table1      = rbind(table1, bigtree_ME_row)
table1      = rbind(table1, bigtree_ML_row)

table1
write.table(table1, file="tables/Table1_average_RF.tsv", quote=FALSE, sep="\t", row.names=FALSE)

Method,Unit.25,Unit.1,Difference
<chr>,<chr>,<chr>,<chr>
ME_SM,0.24 sd 0.15,0.47 sd 0.13,0.23
ME_ST,0.23 sd 0.13,0.47 sd 0.13,0.24
ML_SM,0.19 sd 0.14,0.44 sd 0.13,0.25
ML_ST,0.2 sd 0.12,0.44 sd 0.13,0.24
ME_BigTree,0.25 sd 0.16,Na,Na
ML_BigTree,0.18 sd 0.15,Na,Na


<a id="subchapter-6_2-Figure-2"></a>

### Subchapter 6.2 Figure 2

TODO add description

In [8]:
# Set plot dimensions and resolution for inline display
options(repr.plot.width = 6, repr.plot.height = 5, repr.plot.res = 300)

# Load the necessary data
# Reading in average RF values between reference big tree ME/ML from a file computed in chapter 5.
df_bigtree_ME = read.table("tables/paralog_ME_vs_true_tree.tsv", header=F)
df_bigtree_ML = read.table("tables/paralog_ML_vs_true_tree.tsv", header=F)
rf_df_bigtree_ME_mean = colMeans(df_bigtree_ME)
rf_df_bigtree_ML_mean = colMeans(df_bigtree_ML)

# Reading in average RF values between reference supermatrix and supertree
df_new = read.table("../analysis/tables/avg_RF_between_ref_supermatrix_and_supertree.tsv", header = TRUE)

# Calculate column-wise means of the loaded data
rf_df_new_means = colMeans(df_new)

# Reshape the means data into a long format
rf_df_new_means_melted = melt(rf_df_new_means)

# Rename the first column to "RF" for clarity
colnames(rf_df_new_means_melted)[1] = "RF"

# Add a column indicating the topologies (25 levels, repeated 4 times for each method)
rf_df_new_means_melted$Topology = c(rep(colnames(df_new)[1:25], 4))

# Add a column specifying the method (Supermatrix-ME, Supertree-ME, Supermatrix-ML, Supertree-ML)
rf_df_new_means_melted$Method = c(rep("Supermatrix-ME", 25), rep("Supertree-ME", 25),
                                  rep("Supermatrix-ML", 25), rep("Supertree-ML", 25))

# Sort the data by topology using mixedorder
rf_df_new_means_melted = rf_df_new_means_melted[mixedorder(rf_df_new_means_melted$Topology), ]

# crate the x axis label names
x_axis_labels = gsub("ME_SM_", "", colnames(df_new)[1:25])

# Modify the Topology column to match the cleaned x-axis labels
rf_df_new_means_melted$Topology = rep(x_axis_labels, each = 4)  # Repeat each value per method (4 times)

# Define custom colors for each Method
custom_colors = c("Supertree-ME" = "#7CB6E2",     # Light blue 
                  "Supermatrix-ME" = "#2F739B",   # Dark blue
                  "Supertree-ML" = "#E5A23D",     # Orange/yellow
                  "Supermatrix-ML" = "#D05159")   # Red

# Create the line plot
p = ggplot(rf_df_new_means_melted, aes(x = factor(Topology, levels = x_axis_labels), 
                                       y = RF, group = Method, color = Method)) + 
    geom_line() +  # Add lines connecting the points by method
    geom_point(shape = 16) +  # Use shape 16 (solid dot) for all points

    # Add a horizontal dashed line representing the average RF of ME big tree
    geom_hline(yintercept = rf_df_bigtree_ME_mean, 
               linetype = "dashed", color = "#606060") +
    
    # Annotate the plot with a label for the average bigtree ME RF line
    annotate("text", x = 18, y = rf_df_bigtree_ME_mean + 0.025, 
             label = "Avg RF between BigTree-ME and true topology", 
             size = 3, color = "#606060") +

    # Add a horizontal dashed line representing the average RF of ML big tree
    geom_hline(yintercept = rf_df_bigtree_ML_mean, 
               linetype = "dashed", color = "#1E7005") +
    
    # Annotate the plot with a label for the average bigtree ML RF line
    annotate("text", x = 12, y = rf_df_bigtree_ML_mean - 0.025, 
             label = "Avg RF between BigTree-ML and true topology", 
             size = 3, color = "#1E7005") +

    # Apply a light theme for better readability
    theme_light() + 

    # Adjust x-axis text to be tilted at 45 degrees and align to the right
    theme(axis.text.x = element_text(angle = 45, hjust = 1), 
          legend.position = "right",
          axis.title.x = element_text(face = "bold"),  
          axis.title.y = element_text(face = "bold"))  + 
    
    # Label the axes
    ylab("Normalized RF") + 
    xlab("Topology") + 

    # Set y-axis limits between 0 and 0.5
    scale_y_continuous(limits = c(0, 0.5)) +

    # Customize color and shape scales for the different methods
    scale_color_discrete(breaks = c("Supertree-ME", "Supermatrix-ME", "Supertree-ML", "Supermatrix-ML")) + 
    scale_shape_discrete(breaks = c("Supertree-ME", "Supermatrix-ME", "Supertree-ML", "Supermatrix-ML")) +
    scale_color_manual(values = custom_colors)  # Apply custom colors

# Save the plot to a file with high resolution
ggsave(filename = "figures/RF_lineplot_figure2.png", plot = p, dpi = "retina")

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Saving 7 x 7 in image


<a id="chapter-7-Running-times"></a>

## Chapter 7 - Running times

TODO add description 

In [9]:
# read the trace files for running times
for (run in run_dirs) {
    print(basename(run))

    # each main.nf output run has a trace file repoprting various information on the run itself
    tracefile_path = file.path(run, "trace_cpu_time.txt")
    tracefile      = read.table(tracefile_path, header = TRUE)
    
    # here we parse the trace file to retain only the processes that we want to take the running time (realtime) from
    tracefile_mod  = tracefile[grep("simulated_data:run_phylo_ML_supermatrix_aln_sim|simulated_data:only_concatenate_aln_sim|simulated_data:run_phylo_ML_full_aln_sim", tracefile$native_id), c("name", "native_id", "realtime")]
    
    # transforming the above table to one that have the process names (simulated_data:run_phylo_ML_supermatrix_aln_sim ecc..) as column names, 
    # the rows will be still the family names (working as ID) and the values in each column will be the running times associated to the column name (process name) for that given family.
    tracefile_mod_wide = spread(tracefile_mod, native_id, realtime)

    # creating a new last column called Supermatrix_ML that has aas values the running times of concatenation plus ML tree computation (on concatenated MSA) 
    tracefile_mod_wide$Supermatrix_ML = as.numeric(tracefile_mod_wide$`simulated_data:only_concatenate_aln_sim`) + as.numeric(tracefile_mod_wide$`simulated_data:run_phylo_ML_supermatrix_aln_sim`)

    # retain only the family name, BIgTree ML running time and SM + ML running time
    tracefile_mod_wide = tracefile_mod_wide[,c(1,3,5)]

    # call the second column name Big_Tree and set their value to numeric type
    colnames(tracefile_mod_wide)[2] = "Big_Tree"
    tracefile_mod_wide$Big_Tree = as.numeric(tracefile_mod_wide$Big_Tree)

    # adding a first column that groups families by the run of the main.nf
    tracefile_mod_wide$family_group = rep(basename(run), 100)

    # add tables toghether raw-wise
    if(run == run_dirs[1]) {
        fin_tracefile_mod_wide = tracefile_mod_wide
    } else {
        fin_tracefile_mod_wide = rbind(fin_tracefile_mod_wide, tracefile_mod_wide)
    }
}

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species"


In [10]:
# TODO make nicer and update with whatever is changed above

# make the means of all (500) family running time for the BigTree and ML + SM. Bring them to seconds (/1000) and round the digits to 2
round(colMeans(fin_tracefile_mod_wide[, c(2, 3)])/1000,digits = 2)

# compute the median for the BigTree approach and the  ML + SM.
round(median(fin_tracefile_mod_wide$Big_Tree)/1000,digits = 2)
round(median(fin_tracefile_mod_wide$Supermatrix_ML)/1000,digits = 2)

Big_Tree Supermatrix_ML 
        638.12          67.76

[1] 588.3

[1] 63.85

<a id='subchapter-7_1-Figure-3'></a>

### Subchapter 7.1 - Figure 3

TODO add description if necessary.

In [11]:
# TODO make this into the section of custom functions.
# Function to assign new values based on a mapping provided by the user
assign_mapped_values <- function(df, column_name, mapping, new_col_name) {
  # Check if mapping is a named vector or a list
  if (!is.vector(mapping) || is.null(names(mapping))) {
    stop("Mapping must be a named vector or list.")
  }
  
  # Get the unique values from the specified column
  unique_values = unique(df[[column_name]])
  
  # Check if the mapping contains all unique values
  if (!all(unique_values %in% names(mapping))) {
    stop("Mapping must include all unique values from the specified column.")
  }
  
  # Initialize a vector to store the assigned values
  assigned_values = vector("character", nrow(df))
  
  # Assign values based on the mapping
  for (i in seq_along(unique_values)) {
    assigned_values[df[[column_name]] == unique_values[i]] <- mapping[names(mapping) == unique_values[i]]
  }
  
  # Add the assigned values to the original dataframe
  df[[new_col_name]] = assigned_values
  
  return(df)
}

In [19]:
#
# Cpu time vs Method (SM + ML and Bigtree) divided per group (by sequence length)
#

# assign to each family_group a mapping values  (as new column). basically insert the length of the sequences in each family as an additional column in the dataframe.
# Remember each family in the same group have the same length of sequence generated, the MSA though might differ in length due to gaps.
mapping = c("rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1\nlen = 32",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2\nlen = 46",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3\nlen = 50",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4\nlen = 60",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5\nlen = 82")
fin_tracefile_mod_reassigned = assign_mapped_values(fin_tracefile_mod_wide, "family_group", mapping, "length")

# reorder the columns to have the numerical values at the end
fin_tracefile_mod_reassigned = fin_tracefile_mod_reassigned[, c("name", "family_group", "length", "Big_Tree", "Supermatrix_ML")]

# It takes a table of all families as rows (500) and it goes to a table of 1000 (500 BigTree + 500 SM + ML) where the running times instead of being separated into two different coluns now they are in the same one.
# name + group columns will work as unique ID.  group is the directory name of the main.nf run/s, while name is the name of the family. 
fin_tracefile_mod_melted           = melt(fin_tracefile_mod_reassigned)


# transform running times from millisecond to seconds. And rename the column names.
fin_tracefile_mod_melted$value     = fin_tracefile_mod_melted$value / 1000
colnames(fin_tracefile_mod_melted) = c("name", "family_group", "length", "Method", "Time")

# create a table with 5 pairs of boxplot. Group1, 2.. ecc.. each group has 100 families and two running time (realtime) mesuraments for the BigTree and SM + ML approaches.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_melted, aes(x=length, y=Time, fill=Method)) +
    geom_boxplot(show.legend = TRUE) +
    theme_light() +
    labs(x = "sequence length",
       y = "CPU time (sec)",
       fill = "Method") +

    # Adjust x-axis text to be tilted at 45 degrees and align to the right
    theme(
        axis.title.x = element_text(face = "bold"),  
        axis.title.y = element_text(face = "bold")
    ) +

    # Manually set colors for the two methods "#1E7005" bright green and "#D05159" redish
    scale_fill_manual(
        values = c("Big_Tree" = "#1E7005", "Supermatrix_ML" = "#D05159"),
        labels = c("BigTree-ML", "Supermatrix-ML")
    )

ggsave(filename="figures/CPU_time_per_group_BigTree_SM_ML_figure3.png", plot=p, dpi = "retina")

Using name, family_group, length as id variables

Saving 7 x 7 in image


### Suplementary Fig todo put number of supplementary figure

In [20]:
#
# Cpu time vs Method ratio (Bigtree / SM + ML ) boxplot of each family ratio by group (x axis groups y ratio Big/SM) 
# 

# start by taking the already present info for the families and their grouping from fig 2 cell and compute ratio Bigtree / SM + ML
# Create a new column with the division of Big_Tree by Supermatrix_ML running times for each family (500)
# and then remove the original columns Big_Tree Supermatrix_ML
fin_tracefile_mod_ratio = fin_tracefile_mod_reassigned %>%
  mutate(fold_ratio = Big_Tree / Supermatrix_ML) %>%
  select(-Big_Tree, -Supermatrix_ML)

# create a table with 5 boxplots. Group1, 2.. ecc.. each group has 100 families and  the ratio of two running time (realtime) mesuraments for the BigTree and SM + ML approaches.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_ratio, aes(x=length, y=fold_ratio)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    labs(x = "sequence length",
       y = "CPU time fold ratio (BigTree / Supermatrix_ML)")

# TODO add subplot number to filename
ggsave(filename="figures/CPU_time_per_group_BigTree_SM_ML_ratio.png", plot=p, dpi = "retina")

Saving 7 x 7 in image


<a id="chapter-8-Bootstrap-analysis"></a>

## Chapter 8 - Bootstrap analysis

TODO add decriptoion

In [4]:
#
# ME + SM block
#


# Create the dataframe that will contain all bs values and other info like presence in reference tree
BS_df_ME_SM = data.frame(
  group = character(), 
  family = character(), 
  unit = integer(), 
  sample = integer(),
  node = integer(),
  BS = numeric(), 
  ref_presence = numeric()  # Using integer type for 0 and 1 values
)

for (run in run_dirs) {
    print(basename(run))
    
    # get al the family subdirectories in each mqin.nf output run dir
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = 0

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # work at family level to get the BS value
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 10 == 0) {
            print(m)
        }

        # read the file that contains the length of the MSA (number of columns) of the input MSA of main.nf for that given family.
        bs_file = file.path(m, "results/rerun", "BS.dat")
        bs      = read.table(bs_file)
        myBS    = 32                       # set manually because that is the scaling factor for the smallest of the groups. not dynamical anymore
        #myBS    = bs$V1

        # read the reference tree (gene family tree) the one set as thruth and on which the sequences were generated along.
        ref_tree_file = file.path(run, paste0(basename(m), ".nwk"))
        ref_tree      = read.tree(ref_tree_file, keep.multi=FALSE)

        # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
        # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of one species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
        ref_subtree   = keep.tip(ref_tree, as.character(ref_codefile$V1))

        # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
        for (j in 1:length(ref_subtree$tip.label)) { 
            ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
        }

        # Unroot the tree and put into a list
        ref_paralog_tree = unroot(ref_subtree)
      
        # do a for loop on the number of species (25) for reading all trees of a given unit (from unit1 to unit25).
        for (k in 1:length(orgs$V1)) {

            # read the file that holds all ME SM trees for a given unit. it holds 10 trees one for each sample/reppliceate.
            all_unit_trees_ME_SM_file  = file.path(m, "results/rerun", paste0("unit_", k, "_all.nwk"))
            all_unit_trees_ME_SM       = read.tree(file = all_unit_trees_ME_SM_file, keep.multi = TRUE)
            all_unit_trees_ME_SM       = unroot(all_unit_trees_ME_SM)

            BS_mean_unit_ME_SM = c()
            
            # for loop on the number of trees in all_unit_trees_ME_SM (10). 
            for (i in 1:length(all_unit_trees_ME_SM)) {

                # Each unit_k_sample_i_rep.trees file contains as many trees as number of columns MSA gene family trees. This are all bootstrap replicate trees for the corresponding ME tree in all_unit_trees_ME_SM. 
                # This represents the variability in tree structure across the bootstrap replicates.
                all_bs_rep_tree_ME_SM_file = file.path(m, "results/rerun", paste0("unit_", k, "_sample_", i, "_rep.trees"))
                all_bs_rep_tree_ME_SM      = read.tree(file = all_bs_rep_tree_ME_SM_file, keep.multi = TRUE)
                all_bs_rep_tree_ME_SM      = unroot(all_bs_rep_tree_ME_SM)

                # get only the first 32 of them. So that each group has the same number of replicate trees in it. 
                # TODO remove this workaround variable number of trees
                all_bs_rep_tree_ME_SM = all_bs_rep_tree_ME_SM[1:32]

                # calculates the bootstrap support for each clade in the tree by counting how often each clade (node) in the original tree (all_unit_trees_ME_SM[[i]]) is found in the bootstrap replicate trees (all_bs_rep_tree_ME_SM). 
                # The [-1] removes the root node’s support value, which is usually not meaningful in unrooted trees. 
                # This is a list of 12 ( 15 tips -3, 13 internal nodes for unrooted tree and the first clade is the tree itself). because each tree has 15 tips being a paralog tree. Like this by design choiche.0
                BS_all_unit_trees_ME_SM         = prop.clades(all_unit_trees_ME_SM[[i]], all_bs_rep_tree_ME_SM)[-1]
                
                # check now if a clade in the unit tree of interest is also present in the paralog tree. 1 if present NA if missing.
                # this list and the one above are syncronyzed. meaning the first element of both point to the same clade
                presence_of_clade_in_ref_ME_SM = prop.clades(all_unit_trees_ME_SM[[i]], ref_paralog_tree)[-1]

                # Any NA values in all_trees_BS (indicating clades absent in the bootstrap replicates) are replaced with 0.             
                BS_all_unit_trees_ME_SM[is.na(BS_all_unit_trees_ME_SM)]               = 0
                presence_of_clade_in_ref_ME_SM[is.na(presence_of_clade_in_ref_ME_SM)] = 0

                # Then, the mean of these clade support values is calculated and stored.
                val = mean((as.numeric(BS_all_unit_trees_ME_SM)))
                BS_mean_unit_ME_SM = c(BS_mean_unit_ME_SM, val)

                # Each clade’s support value is scaled by dividing by myBS (which is set to the total number of bootstrap replicates -> number of columns MSA gene family trees).
                # add the clade values to the overall dataframe, teh id of each clade will be the concatenation of the first 5 columns.
                vector3 = 1:12
                tmp_df  = data.frame(
                      group = rep(basename(run), 12),
                      family = rep(m, 12),
                      unit = rep(k, 12),
                      sample = rep(i, 12),
                      node = vector3,
                      BS = c(as.numeric(BS_all_unit_trees_ME_SM) / myBS), # This scales the values to a proportion (0 to 1) for each clade.
                      ref_presence = presence_of_clade_in_ref_ME_SM
                    )

                # add the new block to the overall dataframe
                BS_df_ME_SM = rbind(BS_df_ME_SM, tmp_df)
            }
        }
    }
}

# save as gizepp so it can be pushed to github otherwise is too big.
write.table(BS_df_ME_SM, gzfile("tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), quote=FALSE, sep="\t", row.names=FALSE)

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_010.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_020.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_030.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_1

In [3]:
#
# ML + SM block (like cell above)
#


# Create the dataframe that will contain all bs values and other info like presence in reference tree
BS_df_ML_SM = data.frame(
  group = character(), 
  family = character(), 
  unit = integer(), 
  sample = integer(),
  node = integer(),
  BS = numeric(), 
  ref_presence = numeric()  # Using integer type for 0 and 1 values
)

for (run in run_dirs) {
    print(basename(run))
    
    # get al the family subdirectories in each mqin.nf output run dir
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = 0

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # work at family level to get the BS value
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 10 == 0) {
            print(m)
        }

        # read the file that contains the length of the MSA (number of columns) of the input MSA of main.nf for that given family.
        bs_file = file.path(m, "results/rerun", "BS.dat")
        bs      = read.table(bs_file)
        myBS    = 32                       # set manually because that is the scaling factor for the smallest of the groups. not dynamical anymore
        #myBS    = bs$V1

        # read the reference tree (gene family tree) the one set as thruth and on which the sequences were generated along.
        ref_tree_file = file.path(run, paste0(basename(m), ".nwk"))
        ref_tree      = read.tree(ref_tree_file, keep.multi=FALSE)

        # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
        # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of one species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
        ref_subtree   = keep.tip(ref_tree, as.character(ref_codefile$V1))

        # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
        for (j in 1:length(ref_subtree$tip.label)) { 
            ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
        }

        # Unroot the tree and put into a list
        ref_paralog_tree = unroot(ref_subtree)
      
        # do a for loop on the number of species (25) for reading all trees of a given unit (from unit1 to unit25).
        for (k in 1:length(orgs$V1)) {

            # read the file that holds all ML SM trees for a given unit. it holds 10 trees one for each sample/reppliceate.
            all_unit_trees_ML_SM_file  = file.path(m, "results/rerun", paste0("unit_", k, "_all_raxml.nwk"))
            all_unit_trees_ML_SM       = read.tree(file = all_unit_trees_ML_SM_file, keep.multi = TRUE)
            all_unit_trees_ML_SM       = unroot(all_unit_trees_ML_SM)

            BS_mean_unit_ML_SM = c()
            
            # for loop on the number of trees in all_unit_trees_ML_SM (10). 
            for (i in 1:length(all_unit_trees_ML_SM)) {

                # Each unit_k_sample_i_rep.trees file contains as many trees as number of columns MSA gene family trees. This are all bootstrap replicate trees for the corresponding ML tree in all_unit_trees_ML_SM. 
                # This represents the variability in tree structure across the bootstrap replicates.
                all_bs_rep_tree_ML_SM_file = file.path(m, "results/rerun", paste0("unit_", k, "_sample_", i, "_raxml_rep.trees"))
                all_bs_rep_tree_ML_SM      = read.tree(file = all_bs_rep_tree_ML_SM_file, keep.multi = TRUE)
                all_bs_rep_tree_ML_SM      = unroot(all_bs_rep_tree_ML_SM)

                # get only the first 32 of them. So that each group has the same number of replicate trees in it. 
                # TODO remove this workaround variable number of trees
                all_bs_rep_tree_ML_SM = all_bs_rep_tree_ML_SM[1:32]

                # calculates the bootstrap support for each clade in the tree by counting how often each clade (node) in the original tree (all_unit_trees_ML_SM[[i]]) is found in the bootstrap replicate trees (all_bs_rep_tree_ML_SM). 
                # The [-1] removes the root node’s support value, which is usually not meaningful in unrooted trees. 
                # This is a list of 12 ( 15 tips -3, 13 internal nodes for unrooted tree and the first clade is the tree itself). because each tree has 15 tips being a paralog tree. Like this by design choiche.0
                BS_all_unit_trees_ML_SM         = prop.clades(all_unit_trees_ML_SM[[i]], all_bs_rep_tree_ML_SM)[-1]
                
                # check now if a clade in the unit tree of interest is also present in the paralog tree. 1 if present NA if missing.
                # this list and the one above are syncronyzed. meaning the first element of both point to the same clade
                presence_of_clade_in_ref_ML_SM = prop.clades(all_unit_trees_ML_SM[[i]], ref_paralog_tree)[-1]

                # Any NA values in all_trees_BS (indicating clades absent in the bootstrap replicates) are replaced with 0.             
                BS_all_unit_trees_ML_SM[is.na(BS_all_unit_trees_ML_SM)]               = 0
                presence_of_clade_in_ref_ML_SM[is.na(presence_of_clade_in_ref_ML_SM)] = 0

                # Then, the mean of these clade support values is calculated and stored.
                val = mean((as.numeric(BS_all_unit_trees_ML_SM)))
                BS_mean_unit_ML_SM = c(BS_mean_unit_ML_SM, val)

                # Each clade’s support value is scaled by dividing by myBS (which is set to the total number of bootstrap replicates -> number of columns MSA gene family trees).
                # add the clade values to the overall dataframe, teh id of each clade will be the concatenation of the first 5 columns.
                vector3 = 1:12
                tmp_df1  = data.frame(
                      group = rep(basename(run), 12),
                      family = rep(m, 12),
                      unit = rep(k, 12),
                      sample = rep(i, 12),
                      node = vector3,
                      BS = c(as.numeric(BS_all_unit_trees_ML_SM) / myBS), # This scales the values to a proportion (0 to 1) for each clade.
                      ref_presence = presence_of_clade_in_ref_ML_SM
                    )

                # add the new block to the overall dataframe
                BS_df_ML_SM = rbind(BS_df_ML_SM, tmp_df1)
            }
        }
    }
}

write.table(BS_df_ML_SM, gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), quote=FALSE, sep="\t", row.names=FALSE)

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_010.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_020.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_030.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_1

<a id='subchapter-8_1-AUC-computation'></a>

### Subchapter 8.1 - AUC computation

TODO add description if necessary.

In [3]:
# figure 4.a

# Reading Bootstrap values for each node and presence of node in family paralog reference tree. 
# This is done for ME and ML + SM, this tables are computed in chapter 8 so that this block does not dipend directly onit once the table is created.
df_bs_ME_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)
df_bs_ML_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# Get unique unit values to loop over. assuming two dataframes have same number of units.
unique_units = unique(df_bs_ME_SM$unit)

# Initialize a dataframe to store AUC values for each unit, with names as "unit_1", "unit_2", etc.
auc_df = data.frame(
    unit      = paste0("unit_", unique_units),
    auc_ME_SM = numeric(length(unique_units)),
    auc_ML_SM = numeric(length(unique_units))
)


# Calculate AUC for each unique unit, using all nodes that belong to it
for (unit_num in unique_units) {
    
    # Subset the data for the current unit ME + SM
    unit_data_ME_SM = subset(df_bs_ME_SM, unit == unit_num)

    # Ensure there are enough positive and negative samples to calculate AUC ME + SM 
    if (length(unique(unit_data_ME_SM$ref_presence)) > 1) {
    
        # Calculate AUC using pROC
        roc_obj_ME_SM              = roc(unit_data_ME_SM$ref_presence, unit_data_ME_SM$BS, quiet = TRUE)
        auc_df$auc_ME_SM[unit_num] = auc(roc_obj_ME_SM)
        
    } else {
    
        # If there's only one class, AUC is undefined, assign NA
        auc_df$auc_ME_SM[unit_num] = NA
    }

    # Subset the data for the current unit ML + SM and repeat above procedure
    unit_data_ML_SM = subset(df_bs_ML_SM, unit == unit_num)

    if (length(unique(unit_data_ML_SM$ref_presence)) > 1) {
        roc_obj_ML_SM              = roc(unit_data_ML_SM$ref_presence, unit_data_ML_SM$BS, quiet = TRUE)
        auc_df$auc_ML_SM[unit_num] = auc(roc_obj_ML_SM)
    } else {
        auc_df$auc_ML_SM[unit_num] = NA
    }
   
}

# Reshape data from wide to long format
auc_df_long = auc_df %>%
  pivot_longer(cols = starts_with("auc"), 
               names_to = "metric", 
               values_to = "auc_value")

# Set the unit column as a factor ordered by mixedorder
auc_df_long$unit = factor(auc_df_long$unit, levels=auc_df$unit[mixedorder(auc_df$unit)])

auc_p = ggplot(auc_df_long, aes(x = unit, y = auc_value, color = metric, shape = metric, group = metric)) +

    # Adjust line thickness if needed
    geom_line(size = 1) + 

    # Adjust point size if needed
    geom_point(size = 3) +  

    # 15: square, 17: triangle
    scale_shape_manual(
        values = c(auc_ME_SM = 15, auc_ML_SM = 17),
        labels = c("Supermatrix-ME", "Supermatrix-ML") 
    ) +

    # Set custom colors "#2F739B" darck blue and "#D05159" redish 
    scale_color_manual(
        values = c(auc_ME_SM = "#2F739B", auc_ML_SM = "#D05159"),
        labels = c("Supermatrix-ME", "Supermatrix-ML")  
    ) +

    # Apply a light theme for better readability
    theme_light() + 

    # Adjust x-axis text to be tilted at 45 degrees and align to the right
    theme(axis.text.x = element_text(angle = 45, hjust = 1), 
        legend.position = c(0.80, 0.20),  # Position legend in the bottom-right corner (x = right, y = bottom)
        legend.margin = margin(10, 10, 10, 10),  # Add a bit of margin around the legend
        axis.title.x = element_text(face = "bold"),  
        axis.title.y = element_text(face = "bold")
    )  + 
    
    # Label the axes
    ylab("ROC AUC of Bootstrap Support") + 
    xlab("Topology") + 

    # Set y-axis limits between 0.5 and 1
    scale_y_continuous(limits = c(0.5, 1)) 

ggsave(filename="figures/BS_lineplot_auc_per_unit_ME_ML_SM_figure4a.png", plot=auc_p, dpi = "retina")

In [41]:
# PLot 4.b

# Reading Bootstrap values for each node and presence of node in family paralog reference tree
df_bs_ME_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)
df_bs_ML_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# Get unique unit assuming two dataframes have the same number of units.
unique_units = unique(df_bs_ME_SM$unit)

# Subset the data for the last (25) unit, ME + SM and ML + SM 
unit_data_ME_SM = subset(df_bs_ME_SM, unit == length(unique_units))
unit_data_ML_SM = subset(df_bs_ML_SM, unit == length(unique_units))

# Calculate AUC and ROC curves for ME + SM
if (length(unique(unit_data_ME_SM$ref_presence)) > 1) {
    unit_25_roc_obj_ME_SM = roc(unit_data_ME_SM$ref_presence, unit_data_ME_SM$BS, quiet = TRUE)
    auc_unit_25_ME_SM = auc(unit_25_roc_obj_ME_SM)
    roc_data_ME_SM = data.frame(
        fpr = 1 - unit_25_roc_obj_ME_SM$specificities,
        tpr = unit_25_roc_obj_ME_SM$sensitivities,
        curve = "ME + SM"
    )
} else {
    roc_data_ME_SM = NULL
    auc_unit_25_ME_SM = NA
}

# Calculate AUC and ROC curves for ML + SM
if (length(unique(unit_data_ML_SM$ref_presence)) > 1) {
    unit_25_roc_obj_ML_SM = roc(unit_data_ML_SM$ref_presence, unit_data_ML_SM$BS, quiet = TRUE)
    auc_unit_25_ML_SM = auc(unit_25_roc_obj_ML_SM)
    roc_data_ML_SM = data.frame(
        fpr = 1 - unit_25_roc_obj_ML_SM$specificities,
        tpr = unit_25_roc_obj_ML_SM$sensitivities,
        curve = "ML + SM"
    )
} else {
    roc_data_ML_SM = NULL
    auc_unit_25_ML_SM = NA
}

# Combine both ROC datasets
roc_data_combined = rbind(roc_data_ME_SM, roc_data_ML_SM)

# Plot both ROC curves in a single plot
p_combined = ggplot(roc_data_combined, aes(x = fpr, y = tpr, color = curve)) +
    geom_line(size = 2) +
    geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "grey", size = 1.5) +
    labs(x = "False Positive Rate", y = "True Positive Rate") +
    scale_color_manual(
        values = c("ME + SM" = "#2F739B", "ML + SM" = "#D05159"),
        name = "Unit_25",
        labels = c(
            paste("\nSupermatrix-ME\nAUC =", round(auc_unit_25_ME_SM, 3)),
            paste("\nSupermatrix-ML\nAUC =", round(auc_unit_25_ML_SM, 3))
        )
    ) +
    theme_light(base_size = 18) +
    theme(
        legend.position = c(0.70, 0.20),
        legend.margin = margin(20, 20, 20, 20),
        legend.text = element_text(size = 16),
        axis.title.x = element_text(face = "bold", size = 18),
        axis.title.y = element_text(face = "bold", size = 18)
    )

# Save the combined plot
ggsave(filename="figures/BS_lineplot_auc_unit_25_ME_ML_SM_combined_figure4b.png", plot=p_combined, dpi = "retina")

Saving 7 x 7 in image


In [56]:
# PLot 4.c

# Reading Bootstrap values for each node and presence of node in family paralog reference tree
df_bs_ME_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)
df_bs_ML_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# get the ammount of unique families (500), the two dataframes are assumed to have same number
unique_family_names = unique(df_bs_ME_SM$family)

# prpare the dataframe for the auc
df_family_auc = data.frame(
    family  = c(),
    group   = c(),
    method  = c(),
    auc_val = c()
)


# for each family compute the auc
for (fam_num in 1:length(unique_family_names)) {

    # get only family node datas for ME + SM
    family_data_ME_SM = subset(df_bs_ME_SM, family == unique_family_names[fam_num])
    
    # calculate the ROC curve and then the AUC
    fam_roc_obj_ME_SM = roc(family_data_ME_SM$ref_presence, family_data_ME_SM$BS, quiet = TRUE)
    auc_fam_ME_SM     = auc(fam_roc_obj_ME_SM)

    # updatet the dataframe with relevant information, changing some fields for readability
    df_family_auc = data.frame(
        family  = c(df_family_auc$family, paste0("family_", fam_num)),
        group   = c(df_family_auc$group, family_data_ME_SM$group[1]),    # a family can only be part of one group 
        method  = c(df_family_auc$method, "Supermatrix-ME"),
        auc_val = c(df_family_auc$auc_val, auc_fam_ME_SM)
    )

    # repeat all of the above for ML + SM
    family_data_ML_SM = subset(df_bs_ML_SM, family == unique_family_names[fam_num])
    
    fam_roc_obj_ML_SM = roc(family_data_ML_SM$ref_presence, family_data_ML_SM$BS, quiet = TRUE)
    auc_fam_ML_SM     = auc(fam_roc_obj_ML_SM)

    df_family_auc = data.frame(
        family  = c(df_family_auc$family, paste0("family_", fam_num)),
        group   = c(df_family_auc$group, family_data_ML_SM$group[1]),    # a family can only be part of one group 
        method  = c(df_family_auc$method, "Supermatrix-ML"),
        auc_val = c(df_family_auc$auc_val, auc_fam_ML_SM)
    )    
}

# Map group names based on provided mapping
groupnames_mapping = list(
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1\nlen = 32",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2\nlen = 46",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3\nlen = 50",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4\nlen = 60",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5\nlen = 82"
)

# Apply the group name mapping
df_family_auc = df_family_auc %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )

# Define colors for methods
method_colors = c("Supermatrix-ME" = "#2F739B", "Supermatrix-ML" = "#D05159")

# Plot using ggplot2
p_family = ggplot(df_family_auc, aes(x = group, y = auc_val, color = method)) +
    geom_boxplot(alpha = 0.6) +  # Adding boxplot layer without outliers
    scale_color_manual(values = method_colors) +  # Set colors based on method
    labs(x = "Sequence length", y = "ROC AUC of Bootstrap Support", color = "Method") +  # Set axis and legend labels
    theme_light() +
    theme(
    axis.title = element_text(face = "bold"),  # Make axis labels bold
    legend.position = "top"
    ) + 
    scale_y_continuous(limits = c(0.5, 1))

ggsave(filename="figures/BS_boxplot_auc_ME_ML_SM_combined_figure4c.png", plot=p_family, dpi = "retina")

Saving 7 x 7 in image
Warning message:
“Removed 5 rows containing non-finite outside the scale range
(`stat_boxplot()`).”


In [16]:
# TODO add fiigure number

# Reading Bootstrap values for each node and presence of node in family paralog reference tree
df_bs_ME_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)
df_bs_ML_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# get the ammount of unique families (500), the two dataframes are assumed to have same number
unique_family_names = unique(df_bs_ME_SM$family)

# prpare the dataframe for the auc for both ME and ML
df_family_auc_ME_SM = data.frame(
    family       = c(),
    group        = c(),
    method       = c(),
    auc_delta    = c(),
    avg_BS_delta = c()
)
df_family_auc_ML_SM = df_family_auc_ME_SM

# for each family compute the auc
for (fam_num in 1:length(unique_family_names)) {

    # get only family node datas for ME + SM for unit1 and unit25
    family_data_ME_SM = subset(df_bs_ME_SM, family == unique_family_names[fam_num])
    unit1_data_ME_SM  = subset(family_data_ME_SM, unit == 1)
    unit25_data_ME_SM = subset(family_data_ME_SM, unit == 25)
    
    # compute mean BS value
    delta_fam_avg_BS_ME_SM =  mean(unit25_data_ME_SM$BS) - mean(unit1_data_ME_SM$BS)
    
    # calculate the ROC curve and then the AUC for unit1. if there are only 1 or 0 in the reference column AUC can not be computed.
    if (length(unique(unit1_data_ME_SM$ref_presence)) > 1) {
        unit1_roc_obj_ME_SM = roc(unit1_data_ME_SM$ref_presence, unit1_data_ME_SM$BS, quiet = TRUE)
        auc_unit1_ME_SM     = auc(unit1_roc_obj_ME_SM)
    # if there are only 1 it means THIS chiunk of nodes was good so it should be rewarded.
    } else if (unique(unit1_data_ME_SM$ref_presence == 1)) {        
        auc_unit1_ME_SM     = 1.0
    # if it where only zeroes the chunk should be punished.
    } else {
        auc_unit1_ME_SM     = 0.5
    }

    # repeat as above for unit25
    if (length(unique(unit25_data_ME_SM$ref_presence)) > 1) {
        unit255_roc_obj_ME_SM = roc(unit25_data_ME_SM$ref_presence, unit25_data_ME_SM$BS, quiet = TRUE)
        auc_unit1_ME_SM     = auc(unit1_roc_obj_ME_SM)
    } else if (unique(unit25_data_ME_SM$ref_presence == 1)) {        
        auc_unit25_ME_SM     = 1.0
    } else {
        auc_unit25_ME_SM     = 0.5
    }

    # make the delta AUC
    delta_fam_auc_ME_SM  = auc_unit25_ME_SM - auc_unit1_ME_SM

    # updatet the dataframe with relevant information, changing some fields for readability
    df_family_auc_ME_SM = data.frame(
        family       = c(df_family_auc_ME_SM$family, paste0("family_", fam_num)),
        group        = c(df_family_auc_ME_SM$group, family_data_ME_SM$group[1]),    # a family can only be part of one group 
        method       = c(df_family_auc_ME_SM$method, "Supermatrix-ME"),
        auc_delta    = c(df_family_auc_ME_SM$auc_delta, delta_fam_avg_BS_ME_SM),
        avg_BS_delta = c(df_family_auc_ME_SM$avg_BS_delta, delta_fam_auc_ME_SM)
    )

    # repeat all of the above for ML + SM
    family_data_ML_SM = subset(df_bs_ML_SM, family == unique_family_names[fam_num])
    unit1_data_ML_SM  = subset(family_data_ML_SM, unit == 1)
    unit25_data_ML_SM = subset(family_data_ML_SM, unit == 25)

    delta_fam_avg_BS_ML_SM =  mean(unit25_data_ML_SM$BS) - mean(unit1_data_ML_SM$BS)

    if (length(unique(unit1_data_ML_SM$ref_presence)) > 1) {
        unit1_roc_obj_ML_SM = roc(unit1_data_ML_SM$ref_presence, unit1_data_ML_SM$BS, quiet = TRUE)
        auc_unit1_ML_SM     = auc(unit1_roc_obj_ML_SM)
    } else if (unique(unit1_data_ML_SM$ref_presence == 1)) {        
        auc_unit1_ML_SM     = 1.0
    } else {
        auc_unit1_ML_SM     = 0.5
    }

    if (length(unique(unit25_data_ML_SM$ref_presence)) > 1) {
        unit255_roc_obj_ML_SM = roc(unit25_data_ML_SM$ref_presence, unit25_data_ML_SM$BS, quiet = TRUE)
        auc_unit1_ML_SM     = auc(unit1_roc_obj_ML_SM)
    } else if (unique(unit25_data_ML_SM$ref_presence == 1)) {        
        auc_unit25_ML_SM     = 1.0
    } else {
        auc_unit25_ML_SM     = 0.5
    }

    delta_fam_auc_ML_SM  = auc_unit25_ML_SM - auc_unit1_ML_SM

    df_family_auc_ML_SM = data.frame(
        family       = c(df_family_auc_ML_SM$family, paste0("family_", fam_num)),
        group        = c(df_family_auc_ML_SM$group, family_data_ML_SM$group[1]),    
        method       = c(df_family_auc_ML_SM$method, "Supermatrix-ML"),
        auc_delta    = c(df_family_auc_ML_SM$auc_delta, delta_fam_avg_BS_ML_SM),
        avg_BS_delta = c(df_family_auc_ML_SM$avg_BS_delta, delta_fam_auc_ML_SM)
    )    
}

# Map group names based on provided mapping
groupnames_mapping = list(
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1 len = 32",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2 len = 46",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3 len = 50",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4 len = 60",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5 len = 82"
)

# Apply the group name mapping
df_family_auc_ME_SM = df_family_auc_ME_SM %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )
df_family_auc_ML_SM = df_family_auc_ML_SM %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )

# Fit linear regression model for ME + SM 
model_ME_SM     = lm(auc_delta ~ avg_BS_delta, data = df_family_auc_ME_SM)
slope_ME_SM     = coef(model_ME_SM)[2]  # Extract slope
intercept_ME_SM = coef(model_ME_SM)[1]  # Extract intercept
r_squared_ME_SM = summary(model_ME_SM)$r.squared  # Extract R-squared value

# Fit linear regression model for ML + SM 
model_ML_SM     = lm(auc_delta ~ avg_BS_delta, data = df_family_auc_ML_SM)
slope_ML_SM     = coef(model_ML_SM)[2]  # Extract slope
intercept_ML_SM = coef(model_ML_SM)[1]  # Extract intercept
r_squared_ML_SM = summary(model_ML_SM)$r.squared  # Extract R-squared value

# Create regression equation string for both ME and ML SM
regression_eq_ME_SM = paste0("R² = ", round(r_squared_ME_SM, 2))
regression_eq_ML_SM = paste0("R² = ", round(r_squared_ML_SM, 2))

# Custom colors for each group, change these as needed
custom_colors = c(
  "group2 len = 46" = "#33a02c",   # Change to your preferred color
  "group1 len = 32" = "#1f78b4",   # Change to your preferred color
  "group3 len = 50" = "#e31a1c",   # Change to your preferred color
  "group4 len = 60" = "#ff7f00",   # Change to your preferred color
  "group5 len = 82" = "#6a3d9a"    # Change to your preferred color
)

# Plot ME
p_ME_auc_bs = ggplot(df_family_auc_ME_SM, aes(x = avg_BS_delta, y = auc_delta, color = group)) +
    geom_point(size = 3, alpha = 0.6) +  # Size of points
    geom_abline(intercept = intercept_ME_SM, slope = slope_ME_SM, color = "black", linetype = "dashed") +  # Add regression line
    annotate("text", x = 0.03, y = 0.65, label = regression_eq_ME_SM, hjust = 0, size = 5) +  # Add regression equation text
    labs(x = "unit25 - unit1 Average node Bootstrap Support", y = "unit25 - unit1 ROC AUC of Bootstrap Support") +  # Custom axis labels
    theme_light() +  # Clean theme
    theme(
    axis.title = element_text(face = "bold", size = 14),  # Bold axis labels, adjustable font size
    axis.text = element_text(size = 12)  # Axis text size
    ) +
    #scale_color_manual(values = custom_colors) +  # Apply custom colors
    guides(color = guide_legend(title = "Supermatrix-ME"))   # Legend with title
    #scale_x_continuous(limits = c(0, 1)) +
    #scale_y_continuous(limits = c(0.5, 1))

# Plot ML
p_ML_auc_bs = ggplot(df_family_auc_ML_SM, aes(x = avg_BS_delta, y = auc_delta, color = group)) +
    geom_point(size = 3, alpha = 0.6) +  # Size of points
    geom_abline(intercept = intercept_ML_SM, slope = slope_ML_SM, color = "black", linetype = "dashed") +  # Add regression line
    annotate("text", x = 0.05, y = 0.57, label = regression_eq_ML_SM, hjust = 0, size = 5) +  # Add regression equation text
    labs(x = "unit25 - unit1 Average node Bootstrap Support", y = "unit25 - unit1 ROC AUC of Bootstrap Support") +  # Custom axis labels
    theme_light() +  # Clean theme
    theme(
    axis.title = element_text(face = "bold", size = 14),  # Bold axis labels, adjustable font size
    axis.text = element_text(size = 12)  # Axis text size
    ) +
    #scale_color_manual(values = custom_colors) +  # Apply custom colors
    guides(color = guide_legend(title = "Supermatrix-ML"))   # Legend with title
    #scale_x_continuous(limits = c(0, 1)) +
    #scale_y_continuous(limits = c(0.5, 1))

ggsave(filename="figures/BS_delta_vs_auc_delta_scatterplot_ME.png", plot=p_ME_auc_bs, dpi = "retina")
ggsave(filename="figures/BS_delta_vs_auc_delta_scatterplot_ML.png", plot=p_ML_auc_bs, dpi = "retina")

Saving 7 x 7 in image
Saving 7 x 7 in image


<a id='subchapter-8_2-MCC-computation'></a>

### Subchapter 8.2 - MCC computation

TODO add description if necessary.

In [44]:
# Define a function to find the optimal threshold that maximizes the Matthews Correlation Coefficient (MCC)
# TODO add to custom function block

get_best_mcc <- function(predicted_values, proven_positives, thresholds){
    
    # Initialize variables to store the best threshold and best MCC found so far
    best_threshold = 0
    best_mcc = -1  # Start with -1 as initial MCC to ensure any positive MCC will replace it
    
    # Loop through each threshold in the provided list to evaluate MCC
    for(threshold in thresholds){
        
        # Apply the threshold to classify predictions as positive or negative
        predicted_labels <- ifelse(predicted_values > threshold, 1, 0)
        predicted_labels = factor(predicted_labels, levels = c(0,1))  # Ensure binary factor levels for confusion matrix
        
        # Generate confusion matrix to calculate TP, TN, FP, and FN
        confusion_matrix = table(predicted_labels, proven_positives)

        # converting them to numeric to handle very large numbers later on in the multiplication step
        tp = as.numeric(confusion_matrix[2,2])  # True Positives
        tn = as.numeric(confusion_matrix[1,1])  # True Negatives
        fp = as.numeric(confusion_matrix[1,2]) # False Positives
        fn = as.numeric(confusion_matrix[2,1])  # False Negatives
        
        # Calculate True Positive Rate (TPR) and False Positive Rate (FPR) for ROC analysis (optional)
        tpr = tp / sum(proven_positives)              # True Positive Rate (Sensitivity)
        fpr = fp / sum(proven_positives == 0)         # False Positive Rate

        # Handle potential NaN values in TPR and FPR by setting them to 0
        if(is.nan(tpr)){
            tpr = 0
        }
        if(is.nan(fpr)){
            fpr = 0
        }

        # Calculate Matthews Correlation Coefficient (MCC) for current threshold
        mcc = (tp*tn - fp*fn) / sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        
        # Handle NaN values in MCC by setting MCC to -1
        if(is.nan(mcc)){
            mcc = -1
        }
        
        # Update best MCC, threshold, and metrics if current MCC is higher than previous best
        if(mcc > best_mcc){
            best_mcc = mcc
            best_threshold = threshold
            best_sensitivity = confusion_matrix[2,2] / sum(proven_positives)     # Sensitivity
            best_specificity = confusion_matrix[1,1] / sum(proven_positives == 0) # Specificity
        }
    }
    
    # Return a list with the best MCC, corresponding threshold, sensitivity, and specificity
    return(list(best_mcc = best_mcc, best_threshold = best_threshold, 
                best_sensitivity = best_sensitivity, best_specificity = best_specificity))
}

In [49]:
# TODO insert plot number

# Reading Bootstrap values for each node and presence of node in family paralog reference tree
df_bs_ME_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)
df_bs_ML_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# get the ammount of unique families (500), the two dataframes are assumed to have same number
unique_family_names = unique(df_bs_ME_SM$family)

# prepare the dataframe for the best MCC values
df_family_mcc = data.frame(
    family         = c(),
    group          = c(),
    method         = c(),
    mcc_val        = c(),
    best_threshold = c()
)

# initialize threshold range and step
thresholds = seq(0.2, 1, 0.01)

# for each family compute the best MCC value
for (fam_num in 1:length(unique_family_names)) {

    # get only family node datas for ME + SM
    family_data_ME_SM = subset(df_bs_ME_SM, family == unique_family_names[fam_num])
    
    # calculate best threshold according to MCC and best MCC value
    fam_mcc_infos_ME_SM      = get_best_mcc(family_data_ME_SM$BS, family_data_ME_SM$ref_presence, thresholds)

    # updatet the dataframe with relevant information, changing some fields for readability
    df_family_mcc = data.frame(
        family         = c(df_family_mcc$family, paste0("family_", fam_num)),
        group          = c(df_family_mcc$group, family_data_ME_SM$group[1]),    # a family can only be part of one group 
        method         = c(df_family_mcc$method, "Supermatrix-ME"),
        mcc_val        = c(df_family_mcc$mcc_val, fam_mcc_infos_ME_SM$best_mcc),
        best_threshold = c(df_family_mcc$best_threshold, fam_mcc_infos_ME_SM$best_threshold)
    )

    # repeat all of the above for ML + SM
    family_data_ML_SM = subset(df_bs_ML_SM, family == unique_family_names[fam_num])
    
    fam_mcc_infos_ML_SM      = get_best_mcc(family_data_ML_SM$BS, family_data_ML_SM$ref_presence, thresholds)

    df_family_mcc = data.frame(
        family         = c(df_family_mcc$family, paste0("family_", fam_num)),
        group          = c(df_family_mcc$group, family_data_ML_SM$group[1]),    # a family can only be part of one group 
        method         = c(df_family_mcc$method, "Supermatrix-ML"),
        mcc_val        = c(df_family_mcc$mcc_val, fam_mcc_infos_ML_SM$best_mcc),
        best_threshold = c(df_family_mcc$best_threshold, fam_mcc_infos_ML_SM$best_threshold)
    )    
}

# Map group names based on provided mapping
groupnames_mapping = list(
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1\nlen = 32",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2\nlen = 46",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3\nlen = 50",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4\nlen = 60",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5\nlen = 82"
)

# Apply the group name mapping
df_family_mcc = df_family_mcc %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )

# Define colors for methods
method_colors = c("Supermatrix-ME" = "#2F739B", "Supermatrix-ML" = "#D05159")

# Plot using ggplot2
p_family_mcc = ggplot(df_family_mcc, aes(x = group, y = best_threshold, color = method)) +
    geom_boxplot(alpha = 0.6) +  # Adding boxplot layer without outliers
    scale_color_manual(values = method_colors) +  # Set colors based on method
    labs(x = "Sequence length", y = "Best BS threshold cut-off according to MCC", color = "Method") +  # Set axis and legend labels
    theme_light() +
    theme(
    axis.title = element_text(face = "bold"),  # Make axis labels bold
    legend.position = "top"
    ) + 
    scale_y_continuous(limits = c(0, 1))

ggsave(filename="figures/BS_boxplot_mcc_best_threshold_ME_ML_SM_combined.png", plot=p_family_mcc, dpi = "retina")

Saving 7 x 7 in image


<a id='subchapter-8_3-Reference-presence-vs-BS'></a>

### Subchapter 8.3 - Reference presence vs BS 

TODO add description if necessary.

In [31]:
# todo add figure number

# Reading Bootstrap values for each node and presence of node in family paralog reference tree
df_bs_ME_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)
df_bs_ML_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# get the ammount of unique families (500), the two dataframes are assumed to have same number
unique_family_names = unique(df_bs_ME_SM$family)

# prpare the dataframe for the auc for both ME and ML
df_family_bs_ref_ME_SM = data.frame(
    family      = c(),
    group       = c(),
    method      = c(),
    percent_ref = c(),
    avg_BS      = c()
)
df_family_bs_ref_ML_SM = df_family_bs_ref_ME_SM

# for each family compute the percentage ref presence and abg bootstrap val
for (fam_num in 1:length(unique_family_names)) {

    # get only family node datas for ME + SM 
    family_data_ME_SM = subset(df_bs_ME_SM, family == unique_family_names[fam_num])
    
    # updatet the dataframe with relevant information, changing some fields for readability
    # computing percentage node presence in reference and average BS 
    df_family_bs_ref_ME_SM = data.frame(
        family       = c(df_family_bs_ref_ME_SM$family, paste0("family_", fam_num)),
        group        = c(df_family_bs_ref_ME_SM$group, family_data_ME_SM$group[1]),    # a family can only be part of one group 
        method       = c(df_family_bs_ref_ME_SM$method, "Supermatrix-ME"),
        percent_ref  = c(df_family_bs_ref_ME_SM$percent_ref, sum(family_data_ME_SM$ref_presence==1)/length(family_data_ME_SM$ref_presence)),
        avg_BS       = c(df_family_bs_ref_ME_SM$avg_BS, mean(family_data_ME_SM$BS))
    )
    
    # repeat for ML
    family_data_ML_SM = subset(df_bs_ML_SM, family == unique_family_names[fam_num])
    df_family_bs_ref_ML_SM = data.frame(
        family       = c(df_family_bs_ref_ML_SM$family, paste0("family_", fam_num)),
        group        = c(df_family_bs_ref_ML_SM$group, family_data_ML_SM$group[1]),    # a family can only be part of one group 
        method       = c(df_family_bs_ref_ML_SM$method, "Supermatrix-ML"),
        percent_ref  = c(df_family_bs_ref_ML_SM$percent_ref, sum(family_data_ML_SM$ref_presence==1)/length(family_data_ML_SM$ref_presence)),
        avg_BS       = c(df_family_bs_ref_ML_SM$avg_BS, mean(family_data_ML_SM$BS))
    )
}

# Map group names based on provided mapping
groupnames_mapping = list(
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1 len = 32",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2 len = 46",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3 len = 50",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4 len = 60",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5 len = 82"
)

# Apply the group name mapping
df_family_bs_ref_ME_SM = df_family_bs_ref_ME_SM %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )
df_family_bs_ref_ML_SM = df_family_bs_ref_ML_SM %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )

# Fit linear regression model for ME + SM 
model_ME_SM     = lm(percent_ref ~ avg_BS, data = df_family_bs_ref_ME_SM)
slope_ME_SM     = coef(model_ME_SM)[2]  # Extract slope
intercept_ME_SM = coef(model_ME_SM)[1]  # Extract intercept
r_squared_ME_SM = summary(model_ME_SM)$r.squared  # Extract R-squared value

# Fit linear regression model for ML + SM 
model_ML_SM     = lm(percent_ref ~ avg_BS, data = df_family_bs_ref_ML_SM)
slope_ML_SM     = coef(model_ML_SM)[2]  # Extract slope
intercept_ML_SM = coef(model_ML_SM)[1]  # Extract intercept
r_squared_ML_SM = summary(model_ML_SM)$r.squared  # Extract R-squared value

# Create regression equation string for both ME and ML SM
regression_eq_ME_SM = paste0("R² = ", round(r_squared_ME_SM, 2))
regression_eq_ML_SM = paste0("R² = ", round(r_squared_ML_SM, 2))

# Custom colors for each group, change these as needed
custom_colors = c(
  "group2 len = 46" = "#33a02c",   # Change to your preferred color
  "group1 len = 32" = "#1f78b4",   # Change to your preferred color
  "group3 len = 50" = "#e31a1c",   # Change to your preferred color
  "group4 len = 60" = "#ff7f00",   # Change to your preferred color
  "group5 len = 82" = "#6a3d9a"    # Change to your preferred color
)

# Plot ME
p_ME_auc_bs = ggplot(df_family_bs_ref_ME_SM, aes(x = avg_BS, y = percent_ref, color = group)) +
    geom_point(size = 3, alpha = 0.6) +  # Size of points
    geom_abline(intercept = intercept_ME_SM, slope = slope_ME_SM, color = "black", linetype = "dashed") +  # Add regression line
    annotate("text", x = 0.03, y = 0.65, label = regression_eq_ME_SM, hjust = 0, size = 5) +  # Add regression equation text
    labs(x = "average Bootstrap Support", y = "% nodes in reference tree") +  # Custom axis labels
    theme_light() +  # Clean theme
    theme(
    axis.title = element_text(face = "bold", size = 14),  # Bold axis labels, adjustable font size
    axis.text = element_text(size = 12)  # Axis text size
    ) +
    #scale_color_manual(values = custom_colors) +  # Apply custom colors
    guides(color = guide_legend(title = "Supermatrix-ME")) +  # Legend with title
    scale_x_continuous(limits = c(0, 1)) +
    scale_y_continuous(limits = c(0, 1))

# Plot ML
p_ML_auc_bs = ggplot(df_family_bs_ref_ML_SM, aes(x = avg_BS, y = percent_ref, color = group)) +
    geom_point(size = 3, alpha = 0.6) +  # Size of points
    geom_abline(intercept = intercept_ML_SM, slope = slope_ML_SM, color = "black", linetype = "dashed") +  # Add regression line
    annotate("text", x = 0.03, y = 0.65, label = regression_eq_ML_SM, hjust = 0, size = 5) +  # Add regression equation text
    labs(x = "average Bootstrap Support", y = "% nodes in reference tree") +  # Custom axis labels
    theme_light() +  # Clean theme
    theme(
    axis.title = element_text(face = "bold", size = 14),  # Bold axis labels, adjustable font size
    axis.text = element_text(size = 12)  # Axis text size
    ) +
    #scale_color_manual(values = custom_colors) +  # Apply custom colors
    guides(color = guide_legend(title = "Supermatrix-ML")) +  # Legend with title
    scale_x_continuous(limits = c(0, 1)) +
    scale_y_continuous(limits = c(0, 1))

ggsave(filename="figures/BS_vs_ref_presence_scatterplot_ME.png", plot=p_ME_auc_bs, dpi = "retina")
ggsave(filename="figures/BS_vs_ref_presence_scatterplot_ML.png", plot=p_ML_auc_bs, dpi = "retina")

Saving 7 x 7 in image
Saving 7 x 7 in image


In [4]:
# TODO put table number
# table binning nodes based on their BS value and computing percentages for ref presence.
# ME + SM 

# Reading Bootstrap values for each node and presence of node in family paralog reference tree
df_bs_ME_SM = read.table(gzfile("../analysis/tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)

# get how many units (25) 
num_units = length(unique(df_bs_ME_SM$unit))

# get how many groups there are (5) each with (100) families. groups define sequence length
num_groups = length(unique(df_bs_ME_SM$group))

# Map group names based on provided mapping
groupnames_mapping = list(
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5"
)

# Apply the group name mapping
df_bs_ME_SM = df_bs_ME_SM %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )

# Create empty data frames with 10 rows (one for each BS bin) to match the output vectors.
df_unit1  = data.frame(matrix(ncol = 0, nrow = 10))
df_unit25 = data.frame(matrix(ncol = 0, nrow = 10))

# Loop through five iterations (one for each group).
for (n in 1:num_groups) {
  
  # Filter data for the specific group and unit for the current iteration.
  unit1_data  = subset(df_bs_ME_SM, group == paste0("group", n) & unit == 1)
  unit25_data = subset(df_bs_ME_SM, group == paste0("group", n) & unit == num_units)
  
  # Initialize empty vectors to hold the binned values for the current iteration.
  unit1_bin_percent_nodes   = c()
  unit1_bin_percent_in_ref  = c()
  unit25_bin_percent_nodes  = c()
  unit25_bin_percent_in_ref = c()
  
  # Create bins from 0.1 to 1.0 in steps of 0.1 and calculate percentages for each bin.
  for (i in seq(0.1, 1.0, by = 0.1)) {
    
    # Filter rows within the BS value range for both unit1 and unit25.
    filtered_rows_unit1  = subset(unit1_data, BS > (i - 0.1) & BS <= i)
    filtered_rows_unit25 = subset(unit25_data, BS > (i - 0.1) & BS <= i)
    
    # Calculate the number of nodes in the bin.
    unit1_bin_percent_nodes  = c(unit1_bin_percent_nodes, length(filtered_rows_unit1$node))
    unit25_bin_percent_nodes = c(unit25_bin_percent_nodes, length(filtered_rows_unit25$node))
    #unit25_bin_percent_nodes = c(unit25_bin_percent_nodes, length(filtered_rows_unit25$node) / length(unit25_data$node))
    
    # Calculate the percentage of nodes in reference out of those in the bin.
    unit1_bin_percent_in_ref  = c(unit1_bin_percent_in_ref, paste0(round(sum(filtered_rows_unit1$ref_presence == 1) / length(filtered_rows_unit1$node) * 100, 2), " %"))
    unit25_bin_percent_in_ref = c(unit25_bin_percent_in_ref, paste0(round(sum(filtered_rows_unit25$ref_presence == 1) / length(filtered_rows_unit25$node)* 100, 2), " %"))
  }
  
  # Add the vectors as new columns in the data frames, naming the columns based on the current group.
  df_unit1[[paste0("group", n, "_BS")]]   = unit1_bin_percent_nodes
  df_unit1[[paste0("group", n, "_ref")]]  = unit1_bin_percent_in_ref
  df_unit25[[paste0("group", n, "_BS")]]  = unit25_bin_percent_nodes
  df_unit25[[paste0("group", n, "_ref")]] = unit25_bin_percent_in_ref
}

# Define custom row labels representing BIN values
row_names = c("0<BS<=0.1", "0.1<BS<=0.2", "0.2<BS<=0.3", "0.3<BS<=0.4", 
               "0.4<BS<=0.5", "0.5<BS<=0.6", "0.6<BS<=0.7", "0.7<BS<=0.8", 
               "0.8<BS<=0.9", "0.9<BS<=1")

# Add row names as the first column of the data frame
df_unit1  = data.frame("Row_Name" = row_names, df_unit1)
df_unit25 = data.frame("Row_Name" = row_names, df_unit25)
                              
# Output the data frames.
df_unit1
df_unit25

# save tables in files
write.table(df_unit1, file="tables/unit_1_per_group_nodes_BS_ref_ME_SM.tsv", quote=FALSE, sep="\t", row.names=FALSE)
write.table(df_unit25, file="tables/unit_25_per_group_nodes_BS_ref_ME_SM.tsv", quote=FALSE, sep="\t", row.names=FALSE)

Row_Name,group1_BS,group1_ref,group2_BS,group2_ref,group3_BS,group3_ref,group4_BS,group4_ref,group5_BS,group5_ref
<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>
0<BS<=0.1,1034,10.93 %,802,12.34 %,664,10.24 %,624,13.62 %,386,14.51 %
0.1<BS<=0.2,1527,16.83 %,1365,18.61 %,1267,19.02 %,1012,20.45 %,714,21.71 %
0.2<BS<=0.3,1711,24.84 %,1592,27.95 %,1549,28.92 %,1336,31.81 %,1038,33.62 %
0.3<BS<=0.4,1672,34.33 %,1627,38.11 %,1589,36.69 %,1478,39.78 %,1296,43.75 %
0.4<BS<=0.5,2004,45.66 %,1911,48.87 %,1998,52.45 %,2012,52.44 %,1886,55.62 %
0.5<BS<=0.6,1179,57.17 %,1290,58.14 %,1359,62.77 %,1321,65.33 %,1313,69.46 %
0.6<BS<=0.7,956,65.79 %,1006,71.77 %,1052,70.06 %,1203,75.23 %,1249,77.66 %
0.7<BS<=0.8,727,74.14 %,842,76.72 %,873,79.73 %,1044,83.72 %,1241,84.93 %
0.8<BS<=0.9,573,81.68 %,711,84.53 %,816,89.71 %,904,90.71 %,1180,91.86 %


Row_Name,group1_BS,group1_ref,group2_BS,group2_ref,group3_BS,group3_ref,group4_BS,group4_ref,group5_BS,group5_ref
<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>
0<BS<=0.1,567,18.52 %,339,15.63 %,279,25.81 %,146,16.44 %,59,28.81 %
0.1<BS<=0.2,1160,28.36 %,895,31.28 %,787,37.99 %,498,26.71 %,315,46.67 %
0.2<BS<=0.3,1607,44 %,1294,43.28 %,1223,50.86 %,859,41.44 %,643,48.99 %
0.3<BS<=0.4,1771,57.88 %,1578,58.68 %,1558,58.92 %,1205,55.1 %,883,60.48 %
0.4<BS<=0.5,2236,70.26 %,2093,73.34 %,2084,73.03 %,2042,71.35 %,1527,73.74 %
0.5<BS<=0.6,1483,81.86 %,1497,85.24 %,1477,85.04 %,1530,85.49 %,1428,82.21 %
0.6<BS<=0.7,1141,88.52 %,1313,90.56 %,1324,92.67 %,1661,93.5 %,1527,92.4 %
0.7<BS<=0.8,886,94.02 %,1166,93.65 %,1183,96.7 %,1573,96.5 %,1619,96.85 %
0.8<BS<=0.9,652,97.85 %,943,97.67 %,1062,98.96 %,1246,99.52 %,1719,99.3 %


In [5]:
# TODO put table number
# table binning nodes based on their BS value and computing percentages for ref presence.
# ML + SM 

# Reading Bootstrap values for each node and presence of node in family paralog reference tree
df_bs_ML_SM = read.table(gzfile("tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# get how many units (25) 
num_units = length(unique(df_bs_ML_SM$unit))

# get how many groups there are (5) each with (100) families. groups define sequence length
num_groups = length(unique(df_bs_ML_SM$group))

# Map group names based on provided mapping
groupnames_mapping = list(
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4",
  "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5"
)

# Apply the group name mapping
df_bs_ML_SM = df_bs_ML_SM %>%
  mutate(
    group = unlist(groupnames_mapping[group])
  )

# Create empty data frames with 10 rows (one for each BS bin) to match the output vectors.
df_unit1  = data.frame(matrix(ncol = 0, nrow = 10))
df_unit25 = data.frame(matrix(ncol = 0, nrow = 10))

# Loop through five iterations (one for each group).
for (n in 1:num_groups) {
  
  # Filter data for the specific group and unit for the current iteration.
  unit1_data  = subset(df_bs_ML_SM, group == paste0("group", n) & unit == 1)
  unit25_data = subset(df_bs_ML_SM, group == paste0("group", n) & unit == num_units)
  
  # Initialize empty vectors to hold the binned values for the current iteration.
  unit1_bin_percent_nodes   = c()
  unit1_bin_percent_in_ref  = c()
  unit25_bin_percent_nodes  = c()
  unit25_bin_percent_in_ref = c()
  
  # Create bins from 0.1 to 1.0 in steps of 0.1 and calculate percentages for each bin.
  for (i in seq(0.1, 1.0, by = 0.1)) {
    
    # Filter rows within the BS value range for both unit1 and unit25.
    filtered_rows_unit1  = subset(unit1_data, BS > (i - 0.1) & BS <= i)
    filtered_rows_unit25 = subset(unit25_data, BS > (i - 0.1) & BS <= i)
    
    # Calculate the percentage of nodes in the bin.
    unit1_bin_percent_nodes  = c(unit1_bin_percent_nodes, length(filtered_rows_unit1$node))
    unit25_bin_percent_nodes = c(unit25_bin_percent_nodes, length(filtered_rows_unit25$node))
    
    # Calculate the percentage of nodes in reference out of those in the bin.
    unit1_bin_percent_in_ref  = c(unit1_bin_percent_in_ref, paste0(round(sum(filtered_rows_unit1$ref_presence == 1) / length(filtered_rows_unit1$node) * 100, 2), " %"))
    unit25_bin_percent_in_ref = c(unit25_bin_percent_in_ref, paste0(round(sum(filtered_rows_unit25$ref_presence == 1) / length(filtered_rows_unit25$node)* 100, 2), " %"))
  }
  
  # Add the vectors as new columns in the data frames, naming the columns based on the current group.
  df_unit1[[paste0("group", n, "_BS")]]   = unit1_bin_percent_nodes
  df_unit1[[paste0("group", n, "_ref")]]  = unit1_bin_percent_in_ref
  df_unit25[[paste0("group", n, "_BS")]]  = unit25_bin_percent_nodes
  df_unit25[[paste0("group", n, "_ref")]] = unit25_bin_percent_in_ref
}

# Define custom row labels representing BIN values
row_names = c("0<BS<=0.1", "0.1<BS<=0.2", "0.2<BS<=0.3", "0.3<BS<=0.4", 
               "0.4<BS<=0.5", "0.5<BS<=0.6", "0.6<BS<=0.7", "0.7<BS<=0.8", 
               "0.8<BS<=0.9", "0.9<BS<=1")

# Add row names as the first column of the data frame
df_unit1  = data.frame("Row_Name" = row_names, df_unit1)
df_unit25 = data.frame("Row_Name" = row_names, df_unit25)
                              
# Output the data frames.
df_unit1
df_unit25

# save tables in files
write.table(df_unit1, file="tables/unit_1_per_group_nodes_BS_ref_ML_SM.tsv", quote=FALSE, sep="\t", row.names=FALSE)
write.table(df_unit25, file="tables/unit_25_per_group_nodes_BS_ref_ML_SM.tsv", quote=FALSE, sep="\t", row.names=FALSE)

Row_Name,group1_BS,group1_ref,group2_BS,group2_ref,group3_BS,group3_ref,group4_BS,group4_ref,group5_BS,group5_ref
<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>
0<BS<=0.1,1031,13 %,813,14.15 %,703,13.09 %,519,13.49 %,252,18.65 %
0.1<BS<=0.2,1470,21.29 %,1283,23.69 %,1109,22.81 %,907,27.78 %,574,26.48 %
0.2<BS<=0.3,1616,30.57 %,1544,32.32 %,1404,33.69 %,1280,33.12 %,920,35.87 %
0.3<BS<=0.4,1636,39.73 %,1631,39.36 %,1625,42.71 %,1490,43.15 %,1249,47.48 %
0.4<BS<=0.5,2022,49.7 %,2016,50.89 %,1975,51.44 %,2035,52.63 %,1853,58.12 %
0.5<BS<=0.6,1248,58.57 %,1237,61.76 %,1317,61.88 %,1449,66.46 %,1376,67.51 %
0.6<BS<=0.7,1002,65.87 %,1073,71.11 %,1117,72.78 %,1263,76.17 %,1376,78.85 %
0.7<BS<=0.8,783,77.65 %,875,78.63 %,993,80.06 %,1064,84.4 %,1266,84.99 %
0.8<BS<=0.9,594,84.34 %,716,87.01 %,800,89.25 %,961,92.4 %,1288,92.08 %


Row_Name,group1_BS,group1_ref,group2_BS,group2_ref,group3_BS,group3_ref,group4_BS,group4_ref,group5_BS,group5_ref
<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>
0<BS<=0.1,438,27.63 %,223,25.11 %,237,25.32 %,92,50 %,44,52.27 %
0.1<BS<=0.2,1043,40.46 %,677,41.36 %,522,37.16 %,359,47.63 %,156,53.85 %
0.2<BS<=0.3,1456,54.33 %,1172,55.8 %,912,53.29 %,723,56.02 %,382,59.42 %
0.3<BS<=0.4,1638,63.49 %,1648,63.17 %,1378,64.08 %,1133,65.75 %,726,63.77 %
0.4<BS<=0.5,2208,74.68 %,2278,74.98 %,2060,75.44 %,1971,75.39 %,1525,72.26 %
0.5<BS<=0.6,1532,83.62 %,1463,85.71 %,1527,83.5 %,1556,86.5 %,1265,81.9 %
0.6<BS<=0.7,1302,90.78 %,1319,92.72 %,1493,91.16 %,1571,93.06 %,1435,92.13 %
0.7<BS<=0.8,1061,96.32 %,1217,96.38 %,1377,94.26 %,1591,97.55 %,1645,97.14 %
0.8<BS<=0.9,812,98.77 %,1019,98.92 %,1230,98.37 %,1454,99.24 %,1935,99.74 %


In [6]:
# TODO add plot number
# plot distribution of nodes in BS bins
# ME + SM

# Reading Bootstrap values for each node and presence of node in family paralog reference tree binned bi BS values
df_unit1_bs_ME_SM  = read.table("tables/unit_1_per_group_nodes_BS_ref_ME_SM.tsv", sep="\t", header = TRUE)
df_unit25_bs_ME_SM = read.table("tables/unit_25_per_group_nodes_BS_ref_ME_SM.tsv", sep="\t", header = TRUE)

# retain only columns needed for plotting
df_unit1_bs_ME_SM  = df_unit1_bs_ME_SM %>% select("Row_Name", "group1_BS", "group5_BS")
df_unit25_bs_ME_SM = df_unit25_bs_ME_SM %>% select("Row_Name", "group1_BS", "group5_BS")

# Add a source column to identify the data frame
df_unit1_bs_ME_SM$source  = "unit1"
df_unit25_bs_ME_SM$source = "unit25"

# Combine the 2 data frames
df_combined = bind_rows(df_unit1_bs_ME_SM, df_unit25_bs_ME_SM)

# Convert Row_Name to numeric bins (midpoints of the bins for plotting)
df_combined = df_combined %>%
  mutate(
    bin = case_when(
      Row_Name == "0<BS<=0.1" ~ 0.05,
      Row_Name == "0.1<BS<=0.2" ~ 0.15,
      Row_Name == "0.2<BS<=0.3" ~ 0.25,
      Row_Name == "0.3<BS<=0.4" ~ 0.35,
      Row_Name == "0.4<BS<=0.5" ~ 0.45,
      Row_Name == "0.5<BS<=0.6" ~ 0.55,
      Row_Name == "0.6<BS<=0.7" ~ 0.65,
      Row_Name == "0.7<BS<=0.8" ~ 0.75,
      Row_Name == "0.8<BS<=0.9" ~ 0.85,
      Row_Name == "0.9<BS<=1" ~ 0.95
    )
  )

# Reshape the data for easier plotting with ggplot2
df_long = df_combined %>%
  pivot_longer(cols = starts_with("group"), names_to = "group", values_to = "num_nodes") %>%
  filter(group %in% c("group1_BS", "group5_BS"))

# Define bin midpoints
bin_midpoints = c(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)

# define y ticks
y_ticks = c(0, 500, 1000, 1500, 2000)

# Plotting
num_nodes_distribution_p = ggplot(df_long, aes(x = bin, y = num_nodes, color = interaction(source, group), group = interaction(source, group))) +
  geom_point(size = 2.2, alpha = 0.5) +
  geom_smooth(method = "loess", se = FALSE, size = 1.5) +
  # Customize colors
  scale_color_manual(values = c("blue", "red", "#1AE6F5", "orange"),
                    labels = c("unit1 group1 len = 32", "unit1 group5 len = 82", "unit25 group1 len = 32", "unit25 group5 len = 82")) +
  # Set x-axis breaks to bin midpoints and y ticks
  scale_x_continuous(breaks = bin_midpoints,
                    limits = c(-0.001, 1.001)) +
  scale_y_continuous(breaks = y_ticks) +
  # Add vertical dashed lines
  geom_vline(xintercept = 0.5, size = 1.5, linetype = "dashed", color = "#808080") +
  geom_vline(xintercept = 0.7, size = 1.5, linetype = "dashed", color = "#00CC66") +
  # Labels and theme customization
  labs(
    x = "bins of Bootstrap value",
    y = "number of nodes",
    color = "Supermatrix-ME"
  ) +
  theme_light() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1), 
    axis.title = element_text(face = "bold"),
    legend.text = element_text(size = 10),  # Adjust this size as desired
    # Control major x-axis grid lines to show only where ticks are
    panel.grid.major.x = element_line(color = "lightgrey"),
    panel.grid.minor.x = element_blank()  # Remove minor grid lines on x-axis
  )
  
ggsave(filename="figures/BS_lineplot_num_nodes_distribution_bin_unit1_25_ME_SM.png", plot=num_nodes_distribution_p, dpi = "retina")

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Saving 7 x 7 in image
`geom_smooth()` using formula = 'y ~ x'


In [7]:
# TODO add plot number
# plot distribution of nodes in BS bins
# ML + SM

# Reading Bootstrap values for each node and presence of node in family paralog reference tree binned bi BS values
df_unit1_bs_ML_SM  = read.table("tables/unit_1_per_group_nodes_BS_ref_ML_SM.tsv", sep="\t", header = TRUE)
df_unit25_bs_ML_SM = read.table("tables/unit_25_per_group_nodes_BS_ref_ML_SM.tsv", sep="\t", header = TRUE)

# retain only columns needed for plotting
df_unit1_bs_ML_SM  = df_unit1_bs_ML_SM %>% select("Row_Name", "group1_BS", "group5_BS")
df_unit25_bs_ML_SM = df_unit25_bs_ML_SM %>% select("Row_Name", "group1_BS", "group5_BS")

# Add a source column to identify the data frame
df_unit1_bs_ML_SM$source  = "unit1"
df_unit25_bs_ML_SM$source = "unit25"

# Combine the 2 data frames
df_combined = bind_rows(df_unit1_bs_ML_SM, df_unit25_bs_ML_SM)

# Convert Row_Name to numeric bins (midpoints of the bins for plotting)
df_combined = df_combined %>%
  mutate(
    bin = case_when(
      Row_Name == "0<BS<=0.1" ~ 0.05,
      Row_Name == "0.1<BS<=0.2" ~ 0.15,
      Row_Name == "0.2<BS<=0.3" ~ 0.25,
      Row_Name == "0.3<BS<=0.4" ~ 0.35,
      Row_Name == "0.4<BS<=0.5" ~ 0.45,
      Row_Name == "0.5<BS<=0.6" ~ 0.55,
      Row_Name == "0.6<BS<=0.7" ~ 0.65,
      Row_Name == "0.7<BS<=0.8" ~ 0.75,
      Row_Name == "0.8<BS<=0.9" ~ 0.85,
      Row_Name == "0.9<BS<=1" ~ 0.95
    )
  )

# Reshape the data for easier plotting with ggplot2
df_long = df_combined %>%
  pivot_longer(cols = starts_with("group"), names_to = "group", values_to = "num_nodes") %>%
  filter(group %in% c("group1_BS", "group5_BS"))

# Define bin midpoints
bin_midpoints <- c(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)

# define y ticks
y_ticks = c(0, 500, 1000, 1500, 2000, 2500)

# Plotting
num_nodes_distribution_p = ggplot(df_long, aes(x = bin, y = num_nodes, color = interaction(source, group), group = interaction(source, group))) +
  geom_point(size = 2.2, alpha = 0.5) +
  geom_smooth(method = "loess", se = FALSE, size = 1.5) +
  # Customize colors
  scale_color_manual(values = c("blue", "red", "#1AE6F5", "orange"),
                    labels = c("unit1 group1 len = 32", "unit1 group5 len = 82", "unit25 group1 len = 32", "unit25 group5 len = 82")) +
  # Set x-axis breaks to bin midpoints and y ticks
  scale_x_continuous(breaks = bin_midpoints,
                    limits = c(-0.001, 1.001)) +
  scale_y_continuous(breaks = y_ticks) +
  # Add vertical dashed lines
  geom_vline(xintercept = 0.5, size = 1.5, linetype = "dashed", color = "#808080") +
  geom_vline(xintercept = 0.7, size = 1.5, linetype = "dashed", color = "#00CC66") +
  # Labels and theme customization
  labs(
    x = "bins of Bootstrap value",
    y = "number of nodes",
    color = "Supermatrix-ML"
  ) +
  theme_light() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1), 
    axis.title = element_text(face = "bold"),
    legend.text = element_text(size = 10),  # Adjust this size as desired
    # Control major x-axis grid lines to show only where ticks are
    panel.grid.major.x = element_line(color = "lightgrey"),
    panel.grid.minor.x = element_blank()  # Remove minor grid lines on x-axis
  )

ggsave(filename="figures/BS_lineplot_num_nodes_distribution_bin_unit1_25_ML_SM.png", plot=num_nodes_distribution_p, dpi = "retina")

Saving 7 x 7 in image
`geom_smooth()` using formula = 'y ~ x'


In [8]:
# TODO add plot number
# plot percentage of nodes that are in ref for out of those in the bin
# ME + SM

# Reading Bootstrap values for each node and presence of node in family paralog reference tree binned bi BS values
df_unit1_bs_ME_SM  = read.table("tables/unit_1_per_group_nodes_BS_ref_ME_SM.tsv", sep="\t", header = TRUE)
df_unit25_bs_ME_SM = read.table("tables/unit_25_per_group_nodes_BS_ref_ME_SM.tsv", sep="\t", header = TRUE)

# retain only columns needed for plotting
df_unit1_ref_ME_SM  = df_unit1_bs_ME_SM %>% select("Row_Name", "group1_ref", "group5_ref")
df_unit25_ref_ME_SM = df_unit25_bs_ME_SM %>% select("Row_Name", "group1_ref", "group5_ref")

# Add a source column to identify the data frame
df_unit1_ref_ME_SM$source  = "unit1"
df_unit25_ref_ME_SM$source = "unit25"

# Combine the 2 data frames
df_combined = bind_rows(df_unit1_ref_ME_SM, df_unit25_ref_ME_SM)

# Convert Row_Name to numeric bins (midpoints of the bins for plotting)
df_combined = df_combined %>%
  mutate(
    bin = case_when(
      Row_Name == "0<BS<=0.1" ~ 0.05,
      Row_Name == "0.1<BS<=0.2" ~ 0.15,
      Row_Name == "0.2<BS<=0.3" ~ 0.25,
      Row_Name == "0.3<BS<=0.4" ~ 0.35,
      Row_Name == "0.4<BS<=0.5" ~ 0.45,
      Row_Name == "0.5<BS<=0.6" ~ 0.55,
      Row_Name == "0.6<BS<=0.7" ~ 0.65,
      Row_Name == "0.7<BS<=0.8" ~ 0.75,
      Row_Name == "0.8<BS<=0.9" ~ 0.85,
      Row_Name == "0.9<BS<=1" ~ 0.95
    )
  )

# Convert the columns to numeric and remove the percentage
df_combined = df_combined %>%
  mutate(
    group1_ref = as.numeric(str_remove(group1_ref, " %")),
    group5_ref = as.numeric(str_remove(group5_ref, " %"))
  )

# Reshape the data for easier plotting with ggplot2
df_long = df_combined %>%
  pivot_longer(cols = starts_with("group"), names_to = "group", values_to = "percentage_ref") %>%
  filter(group %in% c("group1_ref", "group5_ref"))

# Define bin midpoints
bin_midpoints <- c(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)

# define y ticks
y_ticks = c(0, 20, 40, 60, 80, 90, 100)
minor_y_breaks = setdiff(seq(0, 100, by = 10), c(90))  # Exclude minor ticks at y=30 and y=70


# Plotting
percentage_nodes_p = ggplot(df_long, aes(x = bin, y = percentage_ref, color = interaction(source, group), group = interaction(source, group))) +
  geom_point(size = 2.2, alpha = 0.5) +
  geom_smooth(method = "loess", se = FALSE, size = 1.5) +
  # Customize colors
  scale_color_manual(values = c("blue", "red", "#1AE6F5", "orange"),
                    labels = c("unit1 group1 len = 32", "unit1 group5 len = 82", "unit25 group1 len = 32", "unit25 group5 len = 82")) +
  # Set x-axis breaks to bin midpoints and y ticks
  scale_x_continuous(breaks = bin_midpoints,
                    limits = c(-0.001, 1.001)) +
  scale_y_continuous(breaks = y_ticks,
                    minor_breaks = minor_y_breaks,
                    limits = c(-0.01, 100.1)) +
  # Add vertical dashed lines
  geom_vline(xintercept = 0.5, size = 1.5, linetype = "dashed", color = "#808080") +
  geom_vline(xintercept = 0.7, size = 1.5, linetype = "dashed", color = "#00CC66") +
  # add orizontal line
  geom_hline(yintercept = 90, size = 0.8, linetype = "dashed", color = "lightgrey") +
  # Labels and theme customization
  labs(
    x = "bins of Bootstrap value",
    y = "% of nodes in bin that are present in reference tree",
    color = "Supermatrix-ME"
  ) +
  theme_light() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1), 
    axis.title = element_text(face = "bold"),
    legend.text = element_text(size = 10),  # Adjust this size as desired
    # Control major x-axis grid lines to show only where ticks are
    panel.grid.major.x = element_line(color = "lightgrey"),
    panel.grid.minor.x = element_blank()  # Remove minor grid lines on x-axis
  )

ggsave(filename="figures/BS_lineplot_percent_nodes_in_bin_in_ref_unit1_25_ME_SM.png", plot=percentage_nodes_p, dpi = "retina")

Saving 7 x 7 in image
`geom_smooth()` using formula = 'y ~ x'


In [9]:
# TODO add plot number
# plot percentage of nodes that are in ref for out of those in the bin
# ML + SM

# Reading Bootstrap values for each node and presence of node in family paralog reference tree binned bi BS values
df_unit1_bs_ML_SM  = read.table("tables/unit_1_per_group_nodes_BS_ref_ML_SM.tsv", sep="\t", header = TRUE)
df_unit25_bs_ML_SM = read.table("tables/unit_25_per_group_nodes_BS_ref_ML_SM.tsv", sep="\t", header = TRUE)

# retain only columns needed for plotting
df_unit1_ref_ML_SM  = df_unit1_bs_ML_SM %>% select("Row_Name", "group1_ref", "group5_ref")
df_unit25_ref_ML_SM = df_unit25_bs_ML_SM %>% select("Row_Name", "group1_ref", "group5_ref")

# Add a source column to identify the data frame
df_unit1_ref_ML_SM$source  = "unit1"
df_unit25_ref_ML_SM$source = "unit25"

# Combine the 2 data frames
df_combined = bind_rows(df_unit1_ref_ML_SM, df_unit25_ref_ML_SM)

# Convert Row_Name to numeric bins (midpoints of the bins for plotting)
df_combined = df_combined %>%
  mutate(
    bin = case_when(
      Row_Name == "0<BS<=0.1" ~ 0.05,
      Row_Name == "0.1<BS<=0.2" ~ 0.15,
      Row_Name == "0.2<BS<=0.3" ~ 0.25,
      Row_Name == "0.3<BS<=0.4" ~ 0.35,
      Row_Name == "0.4<BS<=0.5" ~ 0.45,
      Row_Name == "0.5<BS<=0.6" ~ 0.55,
      Row_Name == "0.6<BS<=0.7" ~ 0.65,
      Row_Name == "0.7<BS<=0.8" ~ 0.75,
      Row_Name == "0.8<BS<=0.9" ~ 0.85,
      Row_Name == "0.9<BS<=1" ~ 0.95
    )
  )

# Convert the columns to numeric and remove the percentage
df_combined = df_combined %>%
  mutate(
    group1_ref = as.numeric(str_remove(group1_ref, " %")),
    group5_ref = as.numeric(str_remove(group5_ref, " %"))
  )

# Reshape the data for easier plotting with ggplot2
df_long = df_combined %>%
  pivot_longer(cols = starts_with("group"), names_to = "group", values_to = "percentage_ref") %>%
  filter(group %in% c("group1_ref", "group5_ref"))

# Define bin midpoints
bin_midpoints <- c(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)

# define y ticks
y_ticks = c(0, 20, 40, 60, 80, 90, 100)
minor_y_breaks = setdiff(seq(0, 100, by = 10), c(90))  # Exclude minor ticks at y=30 and y=70


# Plotting
percentage_nodes_p = ggplot(df_long, aes(x = bin, y = percentage_ref, color = interaction(source, group), group = interaction(source, group))) +
  geom_point(size = 2.2, alpha = 0.5) +
  geom_smooth(method = "loess", se = FALSE, size = 1.5) +
  # Customize colors
  scale_color_manual(values = c("blue", "red", "#1AE6F5", "orange"),
                    labels = c("unit1 group1 len = 32", "unit1 group5 len = 82", "unit25 group1 len = 32", "unit25 group5 len = 82")) +
  # Set x-axis breaks to bin midpoints and y ticks
  scale_x_continuous(breaks = bin_midpoints,
                    limits = c(-0.001, 1.001)) +
  scale_y_continuous(breaks = y_ticks,
                    minor_breaks = minor_y_breaks,
                    limits = c(-0.01, 101)) +
  # Add vertical dashed lines
  geom_vline(xintercept = 0.5, size = 1.5, linetype = "dashed", color = "#808080") +
  geom_vline(xintercept = 0.7, size = 1.5, linetype = "dashed", color = "#00CC66") +
  # add orizontal line
  geom_hline(yintercept = 90, size = 0.8, linetype = "dashed", color = "lightgrey") +
  # Labels and theme customization
  labs(
    x = "bins of Bootstrap value",
    y = "% of nodes in bin that are present in reference tree",
    color = "Supermatrix-ML"
  ) +
  theme_light() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1), 
    axis.title = element_text(face = "bold"),
    legend.text = element_text(size = 10),  # Adjust this size as desired
    # Control major x-axis grid lines to show only where ticks are
    panel.grid.major.x = element_line(color = "lightgrey"),
    panel.grid.minor.x = element_blank()  # Remove minor grid lines on x-axis
  )

ggsave(filename="figures/BS_lineplot_percent_nodes_in_bin_in_ref_unit1_25_ML_SM.png", plot=percentage_nodes_p, dpi = "retina")

Saving 7 x 7 in image
`geom_smooth()` using formula = 'y ~ x'


<br><br><br>
# USEFULL STUFF BUT NOT INCLUDED IN PAPER
<br><br><br>

In [48]:
# related to plot 4

# Load data for ME + SM and ML + SM
df_bs_ME_SM = read.table(gzfile("../analysis/tables/df_bootsrtap_clade_presence_in_ref_ME_SM.tsv.gz"), header = TRUE)
df_bs_ML_SM = read.table(gzfile("../analysis/tables/df_bootsrtap_clade_presence_in_ref_ML_SM.tsv.gz"), header = TRUE)

# define the mapping for numbering of groups
groupnames_mapping = list("rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species"="group1", 
 "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species"="group2",
 "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species"="group3",
 "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species"="group4",
 "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"="group5"
 )

# Rename group values to "group1", "group2", etc., for simplicity
df_bs_ME_SM = df_bs_ME_SM %>%
  mutate( group = unlist(groupnames_mapping[group])
         )
df_bs_ML_SM = df_bs_ML_SM %>%
  mutate( group = unlist(groupnames_mapping[group])
         )

# Get unique units and groups
unique_units  = unique(df_bs_ME_SM$unit)
unique_groups = unique(df_bs_ME_SM$group)

# Initialize a dataframe to store AUC values for each unit-group combination
auc_df = expand.grid(
  unit  = unique_units,
  group = unique_groups,
  stringsAsFactors = FALSE
) %>%
  mutate(
    auc_ME_SM = NA_real_,
    auc_ML_SM = NA_real_
  )

# Calculate AUC for each unit-group combination
for (unit_num in unique_units) {
  for (grp in unique_groups) {
    
    # Subset data for ME + SM
    unit_group_data_ME_SM = subset(df_bs_ME_SM, unit == unit_num & group == grp)
    if (length(unique(unit_group_data_ME_SM$ref_presence)) > 1) {
      roc_obj_ME_SM                                                   = roc(unit_group_data_ME_SM$ref_presence, unit_group_data_ME_SM$BS, quiet = TRUE)
      auc_df$auc_ME_SM[auc_df$unit == unit_num & auc_df$group == grp] = auc(roc_obj_ME_SM)
    } else {
      auc_df$auc_ME_SM[auc_df$unit == unit_num & auc_df$group == grp] = NA
    }
    
    # Subset data for ML + SM
    unit_group_data_ML_SM = subset(df_bs_ML_SM, unit == unit_num & group == grp)
    if (length(unique(unit_group_data_ML_SM$ref_presence)) > 1) {
      roc_obj_ML_SM                                                   = roc(unit_group_data_ML_SM$ref_presence, unit_group_data_ML_SM$BS, quiet = TRUE)
      auc_df$auc_ML_SM[auc_df$unit == unit_num & auc_df$group == grp] = auc(roc_obj_ML_SM)
    } else {
      auc_df$auc_ML_SM[auc_df$unit == unit_num & auc_df$group == grp] = NA
    }
  }
}

# trsform to unit_1 ecc.. unit column
auc_df = auc_df %>%
  mutate(unit = paste0("unit_", unit))

# Reshape data from wide to long format for plotting
auc_df_long = auc_df %>%
  pivot_longer(cols      = starts_with("auc"), 
               names_to  = "metric", 
               values_to = "auc_value") %>%
  mutate(
    method = case_when(
      metric == "auc_ME_SM" ~ "Supermatrix-ME",
      metric == "auc_ML_SM" ~ "Supermatrix-ML"
    )
  )

# Order the 'unit' column based on mixed order for better x-axis presentation
auc_df_long$unit = factor(auc_df_long$unit, levels = unique(auc_df_long$unit)[mixedorder(unique(auc_df_long$unit))])

# Generate the custom labels for the legend plot
custom_label = c(
    "Supermatrix-ME  group1  len = 32",
    "Supermatrix-ML  group1  len = 32",
    "Supermatrix-ME  group2  len = 46",
    "Supermatrix-ML  group2  len = 46",
    "Supermatrix-ME  group3  len = 50",
    "Supermatrix-ML  group3  len = 50",
    "Supermatrix-ME  group4  len = 60",
    "Supermatrix-ML  group4  len = 60",
    "Supermatrix-ME  group5  len = 82",
    "Supermatrix-ML  group5  len = 82"   
)

# designing the color palette of the lineplots, odd numbers are reds and refer to ME, even numbers are blues and refer to ML 
colors_palette = c('#FFB3B3', '#B3CDE3', '#F39A9C', '#92B6D1', '#E78285', '#71A0BF', '#DB696F', '#5089AC', '#D05159', '#2F739B')

# Plotting
auc_p = ggplot(auc_df_long, aes(x = unit, y = auc_value, color = interaction(method, group), shape = interaction(method, group), group = interaction(method, group))) +
  geom_line(size = 0.7) +
  geom_point(size = 2) +
  
  # Set custom shapes and colors for each method-group combination
  scale_shape_manual(
    values = rep(c(15, 17), each = 5),  # Use different shapes for ME (15: square) and ML (17: triangle)
    labels = custom_label
  ) +
  scale_color_manual(
    values = colors_palette,  # Use different colors for ME (dark blue) and ML (reddish)
    labels = custom_label
  ) +
  
  theme_light() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    legend.position = c(0.70, 0.30),
    legend.margin = margin(10, 10, 10, 10),
    axis.title.x = element_text(face = "bold"),
    axis.title.y = element_text(face = "bold")
  ) +
  
  ylab("ROC AUC of Bootstrap Support") +
  xlab("Topology") + 
  scale_y_continuous(limits = c(0.6, 0.9))

# Save the plot
ggsave(filename = "figures/BS_lineplot_auc_per_unit_group_ME_ML_SM.png", plot = auc_p, dpi = "retina")

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Warning message:
“A numeric `legend.position` argument in `theme()` was deprecated in ggplot2
3.5.0.
ℹ Please use the `legend.position.inside` argument of `theme()` instead.”
Saving 7 x 7 in image


In [169]:
#
# Cpu time vs Method (SM + ML and Bigtree)   OLD VERSION
#

# It takes a table of all families as rows (500) and it goes to a table of 1000 (500 BigTree + 500 SM + ML) where the running times instead of being separated into two different coluns now they are in the same one.
# name + group columns will work as unique ID.  group is the directory name of the main.nf run/s, while name is the name of the family. 
fin_tracefile_mod_wide_melted           = melt(fin_tracefile_mod_wide)

# transform running times from millisecond to seconds. And rename the column names.
fin_tracefile_mod_wide_melted$value     = fin_tracefile_mod_wide_melted$value / 1000
colnames(fin_tracefile_mod_wide_melted) = c("name", "family_group", "Method", "Time")

# crate a table with two boxplots (x axis) with Method columns as values and on the y the running times.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel. 
p = ggplot(fin_tracefile_mod_wide_melted,aes(x=Method, y=Time, fill=Method)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    ylab("CPU time (sec)") + xlab("Method")
ggsave(filename="figures/CPU_time_per_method.png", plot=p, dpi = "retina")

Using name, family_group as id variables

Saving 7 x 7 in image


### OLD version all families toghther

### TODO make the following better or remove it

In [48]:
# update the trace matrix with the length of the input msa
library("Biostrings") 
fin_tracefile_mod_with_len = fin_tracefile_mod_wide

# for each family go and read the input MSA (input of main.nf) to get length of MSA (number of columns).
# Iterate over each row of the dataframe
for (i in 1:nrow(fin_tracefile_mod_with_len)) {
    # Construct the full file path using group (directory) and name (filename) to read the input msa
    # TODO make it not depend on the rerun_bla_bla direcrtory
    input_msa_path = file.path(paste0(parent_of_all_dir, "rerun_25_species_corrected_branches"), fin_tracefile_mod_with_len$family_group[i], paste0(gsub("[()]", "", fin_tracefile_mod_with_len$name[i]), ".ma"))

    # read the msa in fasta format and get the length of the first sequence (aka length of the alignment)
    msa_file = readAAStringSet(input_msa_path)
    msa_len  = length(msa_file[[1]])
    
    # add the length to the table 
    fin_tracefile_mod_with_len$length[i] = msa_len
}

In [4]:
#
# Cpu time vs Method (SM + ML and Bigtree) ratio plot of each family (x axis number columns BigMSA) 
# 

# Create a new column with the division of Big_Tree by Supermatrix_ML running times for each family (500)
# and then remove the original columns Big_Tree Supermatrix_ML
fin_tracefile_mod_ratio = fin_tracefile_mod_with_len %>%
  mutate(fold_ratio = Big_Tree / Supermatrix_ML) %>%
  select(-Big_Tree, -Supermatrix_ML)

# create a table with on x axis the length of the MSA (n columns)  and on y the fold ratio (BigTree / SM + ML) of running times. The plot will have now 500 points.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_ratio, aes(x=length, y=fold_ratio)) +
    geom_point(color = 'blue', size = 2, alpha = 0.6) +  # Add points
    theme_light() +
    labs(x = "MSA length (n# columns)",
       y = "fold ratio (BigTree / Supermatrix_ML)")
ggsave(filename="figures/CPU_time_scatterplot_msa_len_BigTree_SM_ML.png", plot=p, dpi = "retina")

ERROR: Error: object 'fin_tracefile_mod_with_len' not found


### TODO the below was just a 10 familoy trial make it go away 

In [109]:
# TODO make this into the section of custom functions.
# Function to convert to milliseconds
convert_to_ms <- function(time_str) {
    # Extract components using regular expressions
    millis = "0"
    seconds = "0"
    minutes = "0"
    if (str_detect(time_str, "\\d+(\\.\\d+)?(?=ms)")) {
        millis <- str_extract(time_str, "\\d+(\\.\\d+)?(?=ms)")
    } else {
        seconds <- str_extract(time_str, "\\d+(\\.\\d+)?(?=s)")
        minutes <- str_extract(time_str, "\\d+(\\.\\d+)?(?=m)")
    }
    
    # Convert each component to milliseconds (coercing to numeric where needed)
    seconds_ms <- as.numeric(seconds) * 1000
    minutes_ms <- as.numeric(minutes) * 60 * 1000
    millis_ms <- as.numeric(millis)
    
    # Sum all components and handle NAs by treating them as 0
    total_ms <- sum(c(seconds_ms, minutes_ms, millis_ms), na.rm = TRUE)
    return(total_ms)
}

In [120]:
for (run in run_dirs) {
    print(basename(run))

    # each main.nf output run has a trace file repoprting various information on the run itself
    tracefile_path = file.path(run, "pipeline_info/execution_trace_2024-09-27_17-39-02.txt")
    tracefile      = read.table(tracefile_path, sep = "\t", header = TRUE)

    # here we parse the trace file to retain only the processes that we want to take the running time (realtime) from
    tracefile_mod = tracefile[grep("simulated_data:run_phylo_ML_supermatrix_aln_sim|simulated_data:only_concatenate_aln_sim|simulated_data:run_phylo_ML_full_aln_sim", tracefile$name), c("name", "realtime")]

    # Use strsplit to split the 'name' column by whitespace
    split_name    = strsplit(as.character(tracefile_mod$name), " ")

    # Create new columns
    tracefile_mod$name       = sapply(split_name, `[`, 2) # First part 
    tracefile_mod$native_id  = sapply(split_name, `[`, 1) # Second part 

    # Reorder columns so 'native_id' becomes the second column
    tracefile_mod = tracefile_mod[, c("name", "native_id", "realtime")]

    # trasform from human readable time to millisecond time
    tracefile_mod$realtime = sapply(tracefile_mod$realtime, convert_to_ms)

    # transforming the above table to one that have the process names (simulated_data:run_phylo_ML_supermatrix_aln_sim ecc..) as column names, 
    # the rows will be still the family names (working as ID) and the values in each column will be the running times associated to the column name (process name) for that given family.
    tracefile_mod_wide = spread(tracefile_mod, native_id, realtime)
    
    # creating a new last column called Supermatrix_ML that has aas values the running times of concatenation plus ML tree computation (on concatenated MSA) 
    tracefile_mod_wide$Supermatrix_ML = as.numeric(tracefile_mod_wide$`simulated_data:only_concatenate_aln_sim`) + as.numeric(tracefile_mod_wide$`simulated_data:run_phylo_ML_supermatrix_aln_sim`)

    # retain only the family name, BIgTree ML running time and SM + ML running time
    tracefile_mod_wide = tracefile_mod_wide[,c(1,3,5)]

    # call the second column name Big_Tree and set their value to numeric type
    colnames(tracefile_mod_wide)[2] = "Big_Tree"
    tracefile_mod_wide$Big_Tree = as.numeric(tracefile_mod_wide$Big_Tree)

    # adding a first column that groups families by the run of the main.nf
    tracefile_mod_wide$group = rep(basename(run), 10)

    # add tables toghether raw-wise
    if(run == run_dirs[1]) {
        fin_tracefile_mod_wide = tracefile_mod_wide
    } else {
        fin_tracefile_mod_wide = rbind(fin_tracefile_mod_wide, tracefile_mod_wide)
    }
}

[1] "results"


In [121]:
# make the means of all (500) family running time for the BigTree and ML + SM. Bring them to seconds (/1000) and round the digits to 2
round(colMeans(fin_tracefile_mod_wide[, c(2, 3)])/1000,digits = 2)

# compute the median for the BigTree approach and the  ML + SM.
round(median(fin_tracefile_mod_wide$Big_Tree)/1000,digits = 2)
round(median(fin_tracefile_mod_wide$Supermatrix_ML)/1000,digits = 2)

Big_Tree Supermatrix_ML 
        195.40          23.47

[1] 196

[1] 21.87

In [122]:

fin_tracefile_mod_wide_melted           = melt(fin_tracefile_mod_wide)

fin_tracefile_mod_wide_melted$value     = fin_tracefile_mod_wide_melted$value / 1000
colnames(fin_tracefile_mod_wide_melted) = c("name","group","Method","Time")

p = ggplot(fin_tracefile_mod_wide_melted,aes(x=Method,y=Time,fill=Method)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    ylab("CPU time (sec)") + xlab("Method")
ggsave(filename="CPU_time_per_method_10_fam.png", plot=p, dpi = "retina")

Using name, group as id variables

Saving 7 x 7 in image
